In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.6/857.6 MB 992.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 14.6 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 19.8 MB/s eta 0:00:0031m50.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 16.6 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 3.0 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 14.9 MB/s eta 0:00:000:00:01 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 9.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 23.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/1

In [1]:
! pip install open3d

In [2]:
!pip install open3d torch torchvision torchaudio numpy

In [3]:

!pip install scikit-learn

In [5]:
import os

import glob

import numpy as np

from sklearn.model_selection import train_test_split



# ✅ Define dataset path

dataset_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/velodyne"



# ✅ Get all .bin files

bin_files = sorted(glob.glob(os.path.join(dataset_path, "*.bin")))



# ✅ Print total files found

print(f"Total Point Cloud Files: {len(bin_files)}")

Total Point Cloud Files: 5985


In [4]:
import os

import glob

import numpy as np

from sklearn.model_selection import train_test_split



# ✅ Define dataset path

dataset_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/label_2"



# ✅ Get all .bin files

bin_files = sorted(glob.glob(os.path.join(dataset_path, "*.txt")))



# ✅ Print total files found

print(f"Total labels Files: {len(bin_files)}")

Total labels Files: 5985


In [7]:
import os

# Define dataset paths
VELODYNE_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/velodyne/"
LABEL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/label_2/"

# Get list of file IDs (without extensions)
velodyne_files = set([f.split('.')[0] for f in os.listdir(VELODYNE_PATH) if f.endswith('.bin')])
label_files = set([f.split('.')[0] for f in os.listdir(LABEL_PATH) if f.endswith('.txt')])

# Find extra files in Velodyne
extra_velodyne_files = velodyne_files - label_files
extra_label_files = label_files - velodyne_files

print(f"Total Velodyne Files: {len(velodyne_files)}")
print(f"Total Label Files: {len(label_files)}")
print(f"Extra Velodyne Files: {extra_velodyne_files}")
print(f"Extra Label Files: {extra_label_files}")


Total Velodyne Files: 5985
Total Label Files: 5985
Extra Velodyne Files: set()
Extra Label Files: set()


In [6]:
import os

import glob

import numpy as np

from sklearn.model_selection import train_test_split



# ✅ Define dataset path

dataset_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata"



# ✅ Get all .bin files

bin_files = sorted(glob.glob(os.path.join(dataset_path, "*.npy")))



# ✅ Print total files found

print(f"Total labels Files: {len(bin_files)}")

Total labels Files: 5984


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn

# Function to load voxel data
def load_voxel_data(file_path):
    """
    Load voxel data from a file.
    
    Args:
        file_path: Path to the voxel data file.
    
    Returns:
        voxels: (D, H, W) array of voxel data.
    """
    voxels = np.load(file_path)  # Assuming the voxel data is saved as a .npy file
    return voxels

# Define the PointPillars model
class PointPillars(nn.Module):
    def __init__(self):
        super(PointPillars, self).__init__()
        # 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)  # Input channels = 4 (x, y, z, intensity)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for detection head
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 4)  # Output: bounding box (x, y, z, dimensions)
        self.fc3 = nn.Linear(512, 1)  # Output: class score (binary classification)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten
        bbox = self.fc2(x)
        class_score = self.fc3(x)
        return bbox, class_score

# Function to validate the model
def validate_model(model, val_data_dir):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    num_samples = 0

    # Get all voxel files in the validation data directory
    voxel_files = [f for f in os.listdir(val_data_dir) if f.endswith('.npy')]
    
    with torch.no_grad():  # Disable gradient calculation
        for file in voxel_files:
            file_path = os.path.join(val_data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            outputs = model(voxels_tensor)
            
            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score
            
            # Compute loss
            loss_bbox = nn.MSELoss()(outputs[0], target_bbox)
            loss_class = nn.BCEWithLogitsLoss()(outputs[1], target_class)
            total_loss += (loss_bbox + loss_class).item()
            num_samples += 1

    avg_loss = total_loss / num_samples if num_samples > 0 else 0
    print(f'Validation Loss: {avg_loss:.4f}')

# Function to train the model
def train_model(model, train_data_dir, val_data_dir, num_epochs=10):
    criterion_bbox = nn.MSELoss()  # Loss for bounding box regression
    criterion_class = nn.BCEWithLogitsLoss()  # Loss for class score
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Get all voxel files in the training data directory
    voxel_files = [f for f in os.listdir(train_data_dir) if f.endswith('.npy')]
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for file in voxel_files:
            file_path = os.path.join(train_data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            outputs = model(voxels_tensor)
            
            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score
            
            # Compute losses
            loss_bbox = criterion_bbox(outputs[0], target_bbox)
            loss_class = criterion_class(outputs[1], target_class)
            loss = loss_bbox + loss_class
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
        # Validate the model after each epoch
        validate_model(model, val_data_dir)

# Set the paths to your voxelized KITTI dataset
train_data_dir = '/path/to/your/kitti/train/voxeldata'  # Adjust this path
val_data_dir = '/path/to/your/kitti/val/voxeldata'  # Adjust this path

# Create model
model = PointPillars()

# Train the model
train_model(model, train_data_dir, val_data_dir, num_epochs=10)

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn

# Function to load voxel data
def load_voxel_data(file_path):
    """
    Load voxel data from a file.
    
    Args:
        file_path: Path to the voxel data file.
    
    Returns:
        voxels: (D, H, W) array of voxel data.
    """
    voxels = np.load(file_path)  # Assuming the voxel data is saved as a .npy file
    return voxels

# Define the PointPillars model
class PointPillars(nn.Module):
    def __init__(self):
        super(PointPillars, self).__init__()
        # 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)  # Input channels = 4 (x, y, z, intensity)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for detection head
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 4)  # Output: bounding box (x, y, z, dimensions)
        self.fc3 = nn.Linear(512, 1)  # Output: class score (binary classification)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten
        bbox = self.fc2(x)
        class_score = self.fc3(x)
        return bbox, class_score

# Function to train the model
def train_model(model, train_data_dir, num_epochs=10):
    criterion_bbox = nn.MSELoss()  # Loss for bounding box regression
    criterion_class = nn.BCEWithLogitsLoss()  # Loss for class score
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Get all voxel files in the training data directory
    voxel_files = [f for f in os.listdir(train_data_dir) if f.endswith('.npy')]
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for file in voxel_files:
            file_path = os.path.join(train_data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            outputs = model(voxels_tensor)
            
            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score
            
            # Compute losses
            loss_bbox = criterion_bbox(outputs[0], target_bbox)
            loss_class = criterion_class(outputs[1], target_class)
            loss = loss_bbox + loss_class
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Set the paths to your voxelized KITTI dataset
train_data_dir = '/path/to/your/kitti/train/voxeldata'  # Adjust this path

# Create model
model = PointPillars()

# Train the model
train_model(model, train_data_dir, num_epochs=10)

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

# Function to load voxel data
def load_voxel_data(file_path):
    """
    Load voxel data from a file.
    
    Args:
        file_path: Path to the voxel data file.
    
    Returns:
        voxels: (D, H, W) array of voxel data.
    """
    voxels = np.load(file_path)  # Assuming the voxel data is saved as a .npy file
    return voxels
# Set the paths to your voxelized KITTI dataset
train_data_dir = '/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'  # Adjust this path


In [2]:
# Define the PointPillars model
class PointPillars(nn.Module):
    def __init__(self):
        super(PointPillars, self).__init__()
        # 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)  # Input channels = 4 (x, y, z, intensity)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for detection head
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 4)  # Output: bounding box (x, y, z, dimensions)
        self.fc3 = nn.Linear(512, 1)  # Output: class score (binary classification)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)  # Flatten
        bbox = self.fc2(x)
        class_score = self.fc3(x)
        return bbox, class_score

In [4]:

# Function to train the model
def train_model(model, train_data_dir, num_epochs=10):
    criterion_bbox = nn.MSELoss()  # Loss for bounding box regression
    criterion_class = nn.BCEWithLogitsLoss()  # Loss for class score
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Get all voxel files in the training data directory
    voxel_files = [f for f in os.listdir(train_data_dir) if f.endswith('.npy')]
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for file in voxel_files:
            file_path = os.path.join(train_data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            outputs = model(voxels_tensor)
            
            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score
            
            # Compute losses
            loss_bbox = criterion_bbox(outputs[0], target_bbox)
            loss_class = criterion_class(outputs[1], target_class)
            loss = loss_bbox + loss_class
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Set the paths to your voxelized KITTI dataset
train_data_dir = '/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'  # Adjust this path

# Create model
model = PointPillars()

# Train the model
train_model(model, train_data_dir, num_epochs=10)

RuntimeError: Expected 5-dimensional input for 5-dimensional weight [64, 4, 3, 3, 3], but got 3-dimensional input of size [1, 38880, 9] instead

In [5]:
# Function to train the model
def train_model(model, train_data_dir, num_epochs=10):
    criterion_bbox = nn.MSELoss()  # Loss for bounding box regression
    criterion_class = nn.BCEWithLogitsLoss()  # Loss for class score
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Get all voxel files in the training data directory
    voxel_files = [f for f in os.listdir(train_data_dir) if f.endswith('.npy')]

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for file in voxel_files:
            file_path = os.path.join(train_data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor and reshape to 5D for 3D CNN
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32)  # (P, N, C) format
            
            # Ensure correct shape for 3D CNN: (batch, channels, D, H, W)
            voxels_tensor = voxels_tensor.permute(2, 0, 1).unsqueeze(0).unsqueeze(0)  # (1, 1, C, P, N)

            # Forward pass
            outputs = model(voxels_tensor)

            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score

            # Compute losses
            loss_bbox = criterion_bbox(outputs[0], target_bbox)
            loss_class = criterion_class(outputs[1], target_class)
            loss = loss_bbox + loss_class

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



In [7]:
import torch
import os

# Define file paths
train_voxel_path = os.path.join("/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'")  # Update with the actual path
     # Update with the actual path

# Load the saved voxelized data
train_voxel_data = torch.load(train_voxel_path)


# Check data shape
print(f"Train Data Shape: {train_voxel_data.shape}")



FileNotFoundError: [Errno 2] No such file or directory: "/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'"

In [3]:
import numpy as np
import os

def load_voxel_data(file_path):
    """
    Load voxel data from a file.
    
    Args:
        file_path: Path to the voxel data file.
    
    Returns:
        voxels: (D, H, W) array of voxel data.
    """
    voxels = np.load(file_path)  # Assuming the voxel data is saved as a .npy file
    return voxels

In [4]:
import torch
import torch.nn as nn

class VoxelNet(nn.Module):
    def __init__(self):
        super(VoxelNet, self).__init__()
        # 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)  # Input channels = 4 (x, y, z, intensity)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for detection head
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 3)  # Output: x, y, z of bounding box

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

In [5]:
class SSDHead(nn.Module):
    def __init__(self):
        super(SSDHead, self).__init__()
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 4)  # Output: bounding box (x, y, z, dimensions)
        self.fc3 = nn.Linear(512, 1)  # Output: class score (binary classification)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        bbox = self.fc2(x)
        class_score = self.fc3(x)
        return bbox, class_score

In [7]:
import torch
import torch.nn as nn

class VoxelNet(nn.Module):
    def __init__(self):
        super(VoxelNet, self).__init__()
        # 3D Convolutional layers
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=64, kernel_size=3, stride=1, padding=1)  # Input channels = 4 (x, y, z, intensity)
        self.conv2 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv3d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for detection head
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 3)  # Output: x, y, z of bounding box

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

class SSDHead(nn.Module):
    def __init__(self):
        super(SSDHead, self).__init__()
        self.fc1 = nn.Linear(256 * 10 * 10 * 10, 512)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(512, 4)  # Output: bounding box (x, y, z, dimensions)
        self.fc3 = nn.Linear(512, 1)  # Output: class score (binary classification)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        bbox = self.fc2(x)
        class_score = self.fc3(x)
        return bbox, class_score
def train_model(model, data_dir, num_epochs=10):
    criterion_bbox = nn.MSELoss()  # Loss for bounding box regression
    criterion_class = nn.BCEWithLogitsLoss()  # Loss for class score
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Get all voxel files in the data directory
    voxel_files = [f for f in os.listdir(data_dir) if f.endswith('.npy')]
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for file in voxel_files:
            file_path = os.path.join(data_dir, file)
            voxels = load_voxel_data(file_path)  # Load voxel data
            
            # Convert voxels to tensor
            voxels_tensor = torch.tensor(voxels, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            
            # Forward pass
            outputs = model(voxels_tensor)
            
            # Example target (replace with actual target)
            target_bbox = torch.rand(1, 4)  # Random target for bounding box (x, y, z, dimensions)
            target_class = torch.tensor([[1.0]])  # Random target for class score
            
            # Compute losses
            loss_bbox = criterion_bbox(outputs[0], target_bbox)
            loss_class = criterion_class(outputs[1], target_class)
            loss = loss_bbox + loss_class
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Set the path to your voxelized KITTI dataset
data_dir = '/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'  # Adjust this path

# Create model
model = VoxelNet()

# Train the model
train_model(model, data_dir, num_epochs=10)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_11537/999007534.py", line 85, in <module>
    train_model(model, data_dir, num_epochs=10)
  File "/tmp/ipykernel_11537/999007534.py", line 45, in train_model
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/optim/adam.py", line 73, in __init__
    state_sums = []
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/optim/optimizer.py", line 367, in __init__
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/_compile.py", line 26, in inner
ModuleNotFoundError: No module named 'torch._dynamo'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/

In [8]:
import torch
print(torch.__version__)

2.4.1+cu118


In [ ]:

!pip install torch torchvision torchaudio

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels=9, out_channels=64):
        super(PillarFeatureNet, self).__init__()
        
        # MLP layers (PointNet-style)
        self.fc1 = nn.Linear(in_channels, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Linear(64, out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)

    def forward(self, pillars, num_points_per_pillar):
        """
        pillars: (batch_size, max_pillars, max_points, num_features)
        num_points_per_pillar: (batch_size, max_pillars) - Number of points in each pillar
        """
        batch_size, max_pillars, max_points, num_features = pillars.shape

        # Compute mean of each pillar
        pillar_means = pillars[:, :, :, :3].sum(dim=2, keepdim=True) / num_points_per_pillar[:, :, None, None]
        
        # Center each point within the pillar
        centered_pillars = pillars.clone()
        centered_pillars[:, :, :, :3] -= pillar_means  # XYZ - mean XYZ

        # Flatten input for MLP
        pillars_flat = centered_pillars.view(-1, num_features)  # Shape: (batch_size * max_pillars * max_points, num_features)

        # Apply MLP layers
        x = F.relu(self.bn1(self.fc1(pillars_flat)))
        x = self.bn2(self.fc2(x))

        # Reshape back to (batch_size, max_pillars, max_points, out_channels)
        x = x.view(batch_size, max_pillars, max_points, -1)

        # Apply max pooling over points in each pillar
        x = torch.max(x, dim=2)[0]  # Shape: (batch_size, max_pillars, out_channels)

        return x


In [9]:


 

import torch

import torch.nn as nn

import numpy as np

import os



# ✅ Define Pillar Feature Net

class PillarFeatureNet(nn.Module):

    def __init__(self, input_dim=4, output_dim=64):

        super(PillarFeatureNet, self).__init__()

        self.fc = nn.Sequential(

            nn.Linear(input_dim, 64),  

            nn.ReLU(),

            nn.Linear(64, output_dim)

        )



    def forward(self, x):

        return self.fc(x)  # Process each voxel



# ✅ Initialize Feature Extractor

feature_net = PillarFeatureNet()


# ✅ Directory to Save Extracted Features

output_feature_path = r"C:\Users\Alladurgam Shivani\Desktop\3D_OBJ\split\training\pillar_features"

os.makedirs(output_feature_path, exist_ok=True)  # Create directory if not exists



# ✅ Process and Extract Features for Entire Dataset

extracted_features = []

num_saved = 0  # Counter for tracking saved files



for batch_idx, batch in enumerate(dataloader):

    features = feature_net(batch)  # Pass through feature extractor

    extracted_features.append(features.detach().cpu().numpy())



    # ✅ Save Each Batch Separately

    feature_filename = os.path.join(output_feature_path, f"pillar_features_{batch_idx:06d}.npy")

    np.save(feature_filename, extracted_features[-1])

    

    num_saved += 1  # Update Counter

    print(f"✅ Saved Feature File {num_saved}: {feature_filename}")  # Display Each Save



# ✅ Combine All Extracted Features into a Single File (Optional)

extracted_features = np.concatenate(extracted_features, axis=0)

final_feature_file = os.path.join(output_feature_path, "pillar_features_all.npy")

np.save(final_feature_file, extracted_features)



print(f"\n🚀 All Features Extracted & Saved: {final_feature_file}")

print(f"✅ Total Saved Files: {num_saved + 1}")  # Include the final saved file count

NameError: name 'dataloader' is not defined

In [10]:
from torch.utils.data import Dataset, DataLoader

class KittiDataset(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.files = [f for f in os.listdir(data_path) if f.endswith('.bin')]
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file_path = os.path.join(self.data_path, self.files[idx])
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)  # x, y, z, intensity
        return torch.tensor(points, dtype=torch.float32)

# ✅ Define dataset path
dataset_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/velodyne"

# ✅ Create Dataset Instance
dataset = KittiDataset(dataset_path)

# ✅ Create DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: torch.stack(x, dim=0))


In [12]:
import torch
import torch.nn as nn
import numpy as np
import os

# ✅ Define Pillar Feature Net for Feature Extraction
class PillarFeatureNet(nn.Module):
    def __init__(self, input_dim=4, output_dim=64):
        super(PillarFeatureNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)  # Process each voxel

# ✅ Initialize Feature Extractor
feature_net = PillarFeatureNet()

# ✅ Define Output Path for Extracted Features
output_feature_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"
os.makedirs(output_feature_path, exist_ok=True)  # Create directory if not exists

# ✅ Assume voxelized_data is already generated from the voxelization step
# voxelized_data should be a list where each element is a tensor of shape [num_voxels, 4] (x, y, z, intensity)

# Simulated example: Load precomputed voxel data from disk (modify path as needed)
voxel_data_path = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxel_000000.npy"
voxelized_data = np.load(voxel_data_path, allow_pickle=True)  # Assuming voxelized data is saved as .npy

# ✅ Process and Extract Features for Entire Dataset
extracted_features = []
num_saved = 0  # Counter for tracking saved files

for batch_idx, batch in enumerate(voxelized_data):  
    batch_tensor = torch.tensor(batch, dtype=torch.float32)  # Convert numpy to PyTorch tensor
    features = feature_net(batch_tensor)  # Pass through feature extractor
    extracted_features.append(features.detach().cpu().numpy())

    # ✅ Save Each Batch Separately
    feature_filename = os.path.join(output_feature_path, f"pillar_features_{batch_idx:06d}.npy")
    np.save(feature_filename, extracted_features[-1])

    num_saved += 1  # Update Counter
    print(f"✅ Saved Feature File {num_saved}: {feature_filename}")  # Display Each Save

# ✅ Combine All Extracted Features into a Single File (Optional)
extracted_features = np.concatenate(extracted_features, axis=0)
final_feature_file = os.path.join(output_feature_path, "pillar_features_all.npy")
np.save(final_feature_file, extracted_features)

print(f"\n🚀 All Features Extracted & Saved: {final_feature_file}")
print(f"✅ Total Saved Files: {num_saved + 1}")  # Include the final saved file count


FileNotFoundError: [Errno 2] No such file or directory: '/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxel_000000.npy'

In [14]:
import torch
import torch.nn as nn
import numpy as np
import os

# ✅ Define Pillar Feature Net for Feature Extraction
class PillarFeatureNet(nn.Module):
    def __init__(self, input_dim=4, output_dim=64):
        super(PillarFeatureNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)  # Process each voxel

# ✅ Initialize Feature Extractor
feature_net = PillarFeatureNet()

# ✅ Define Paths
voxelized_data_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata"  # Folder containing voxelized files
output_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"  # Folder to save extracted features
os.makedirs(output_feature_folder, exist_ok=True)  # Create output folder if it doesn't exist

# ✅ List All Voxelized Data Files
voxel_files = [f for f in os.listdir(voxelized_data_folder) if f.endswith('.npy')]
voxel_files.sort()  # Sort to maintain order

# ✅ Process and Extract Features for Entire Dataset
num_saved = 0  # Counter for tracking saved files

for file_idx, voxel_file in enumerate(voxel_files):
    voxel_file_path = os.path.join(voxelized_data_folder, voxel_file)
    
    # ✅ Load Voxelized Data from File
    voxel_data = np.load(voxel_file_path, allow_pickle=True)  # Load voxelized point cloud data
    voxel_tensor = torch.tensor(voxel_data, dtype=torch.float32)  # Convert numpy to PyTorch tensor
    
    # ✅ Extract Features
    features = feature_net(voxel_tensor)  # Pass through feature extractor
    
    # ✅ Save Extracted Features
    feature_filename = os.path.join(output_feature_folder, f"pillar_features_{file_idx:06d}.npy")
    np.save(feature_filename, features.detach().cpu().numpy())
    
    num_saved += 1  # Update Counter
    print(f"✅ Saved Feature File {num_saved}: {feature_filename}")  # Display Save Progress

# ✅ Done
print(f"\n🚀 Feature Extraction Completed for {num_saved} Files!")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (33776x9 and 4x64)

In [16]:
voxeldata = np.load(voxel_files[0], allow_pickle=True)
print("Voxel Data Shape:", voxeldata.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'voxel_000000.npy'

In [17]:
import torch
import torch.nn as nn
import numpy as np
import os

# ✅ Define Pillar Feature Net
class PillarFeatureNet(nn.Module):
    def __init__(self, input_dim=9, output_dim=64):  # Adjust input_dim if needed
        super(PillarFeatureNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)  # Process each voxel

# ✅ Initialize Feature Extractor
feature_net = PillarFeatureNet()

# ✅ Define Paths
voxelized_data_folder =  r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata" 
output_feature_folder =  r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"  
os.makedirs(output_feature_folder, exist_ok=True)  

# ✅ List All Voxelized Data Files
voxel_files = [f for f in os.listdir(voxelized_data_folder) if f.endswith('.npy')]
voxel_files.sort()  

# ✅ Process and Extract Features for Entire Dataset
num_saved = 0  

for file_idx, voxel_file in enumerate(voxel_files):
    voxel_file_path = os.path.join(voxelized_data_folder, voxel_file)
    
    # ✅ Load Voxelized Data from File
    voxel_data = np.load(voxel_file_path, allow_pickle=True)  
    
    # ✅ Check and Adjust Shape Dynamically
    num_features = voxel_data.shape[1]  
    if num_features != feature_net.fc[0].in_features:
        print(f"⚠️ Mismatch detected in {voxel_file}. Adjusting input dimension...")
        feature_net = PillarFeatureNet(input_dim=num_features)  # Dynamically adjust input size

    voxel_tensor = torch.tensor(voxel_data, dtype=torch.float32)  
    
    # ✅ Extract Features
    features = feature_net(voxel_tensor)  
    
    # ✅ Save Extracted Features
    feature_filename = os.path.join(output_feature_folder, f"pillar_features_{file_idx:06d}.npy")
    np.save(feature_filename, features.detach().cpu().numpy())
    
    num_saved += 1  
    print(f"✅ Saved Feature File {num_saved}: {feature_filename}")  

# ✅ Done
print(f"\n🚀 Feature Extraction Completed for {num_saved} Files!")


✅ Saved Feature File 1: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000000.npy
✅ Saved Feature File 2: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000001.npy
✅ Saved Feature File 3: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000002.npy
✅ Saved Feature File 4: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000003.npy
✅ Saved Feature File 5: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000004.npy
✅ Saved Feature File 6: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000005.npy
✅ Saved Feature File 7: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000006.npy
✅ Saved Feature File 8: /home/user/Desktop/notee

OSError: 2249344 requested and 1904608 written

In [18]:
import torch
import torch.nn as nn
import numpy as np
import os

# ✅ Define Pillar Feature Net
class PillarFeatureNet(nn.Module):
    def __init__(self, input_dim=9, output_dim=64):  # Adjust input_dim if needed
        super(PillarFeatureNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.fc(x)  # Process each voxel

# ✅ Initialize Feature Extractor
feature_net = PillarFeatureNet()

# ✅ Define Paths
voxelized_data_folder =  r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata" 
output_feature_folder =  r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"  
os.makedirs(output_feature_folder, exist_ok=True)  

# ✅ List All Voxelized Data Files
voxel_files = [f for f in os.listdir(voxelized_data_folder) if f.endswith('.npy')]
voxel_files.sort()  

# ✅ Process and Extract Features for Entire Dataset
num_saved = 0  

for file_idx, voxel_file in enumerate(voxel_files):
    voxel_file_path = os.path.join(voxelized_data_folder, voxel_file)
    
    # ✅ Load Voxelized Data from File
    voxel_data = np.load(voxel_file_path, allow_pickle=True)  
    
    # ✅ Check and Adjust Shape Dynamically
    num_features = voxel_data.shape[1]  
    if num_features != feature_net.fc[0].in_features:
        print(f"⚠️ Mismatch detected in {voxel_file}. Adjusting input dimension...")
        feature_net = PillarFeatureNet(input_dim=num_features)  # Dynamically adjust input size

    voxel_tensor = torch.tensor(voxel_data, dtype=torch.float32)  
    
    # ✅ Extract Features
    features = feature_net(voxel_tensor)  
    
    # ✅ Save Extracted Features
    feature_filename = os.path.join(output_feature_folder, f"pillar_features_{file_idx:06d}.npy")
    np.save(feature_filename, features.detach().cpu().numpy())
    
    num_saved += 1  
    print(f"✅ Saved Feature File {num_saved}: {feature_filename}")  

# ✅ Done
print(f"\n🚀 Feature Extraction Completed for {num_saved} Files!")


✅ Saved Feature File 1: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000000.npy
✅ Saved Feature File 2: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000001.npy
✅ Saved Feature File 3: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000002.npy
✅ Saved Feature File 4: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000003.npy
✅ Saved Feature File 5: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000004.npy
✅ Saved Feature File 6: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000005.npy
✅ Saved Feature File 7: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features/pillar_features_000006.npy
✅ Saved Feature File 8: /home/user/Desktop/notee

In [ ]:
import torch
import numpy as np
import os

# ✅ Define BEV Grid Parameters
GRID_X, GRID_Y = 400, 400  # Adjust based on dataset
FEATURE_DIM = 64  # Output dim from PillarFeatureNet

# ✅ Paths
pillar_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"
pseudo_image_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images"
os.makedirs(pseudo_image_folder, exist_ok=True)

# ✅ List All Feature Files
feature_files = [f for f in os.listdir(pillar_feature_folder) if f.endswith('.npy')]
feature_files.sort()

# ✅ Process Each Feature File and Generate BEV Image
for file_idx, feature_file in enumerate(feature_files):
    feature_path = os.path.join(pillar_feature_folder, feature_file)
    
    # ✅ Load Feature Data
    features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

    # ✅ Initialize Empty BEV Grid
    pseudo_image = np.zeros((FEATURE_DIM, GRID_X, GRID_Y), dtype=np.float32)

    # ✅ Map Features to BEV Grid
    for pillar in features:
        x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual pillar coordinates
        pseudo_image[:, x, y] = pillar  # Assign feature vector to BEV cell

    # ✅ Save Pseudo-Image
    pseudo_image_filename = os.path.join(pseudo_image_folder, f"pseudo_image_{file_idx:06d}.npy")
    np.save(pseudo_image_filename, pseudo_image)

    print(f"✅ Saved Pseudo-Image {file_idx+1}: {pseudo_image_filename}")

# ✅ Done
print(f"\n🚀 Pseudo-Image Generation Completed for {len(feature_files)} Files!")


✅ Saved Pseudo-Image 1: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000000.npy
✅ Saved Pseudo-Image 2: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000001.npy
✅ Saved Pseudo-Image 3: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000002.npy
✅ Saved Pseudo-Image 4: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000003.npy
✅ Saved Pseudo-Image 5: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000004.npy
✅ Saved Pseudo-Image 6: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000005.npy
✅ Saved Pseudo-Image 7: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000006.npy
✅ Saved Pseudo-Image 8: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split

In [2]:
import numpy as np

pseudo_image = np.load(r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000000.npy")
print(pseudo_image.shape)  # Expected Output: (64, 400, 400)


(64, 400, 400)


In [3]:
import torch
import numpy as np
import os

# ✅ Define BEV Grid Parameters
GRID_X, GRID_Y = 400, 400  # Adjust based on dataset
FEATURE_DIM = 64  # Output dim from PillarFeatureNet

# ✅ Paths
pillar_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"
pseudo_image_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images"
os.makedirs(pseudo_image_folder, exist_ok=True)

# ✅ List All Feature Files
feature_files = [f for f in os.listdir(pillar_feature_folder) if f.endswith('.npy')]
feature_files.sort()

# ✅ Check Already Processed Files
processed_files = {f"pseudo_image_{i:06d}.npy" for i in range(433)}  # Already saved 432 images

# ✅ Process Remaining Feature Files
for file_idx, feature_file in enumerate(feature_files):
    pseudo_image_filename = f"pseudo_image_{file_idx:06d}.npy"
    pseudo_image_path = os.path.join(pseudo_image_folder, pseudo_image_filename)
    
    # Skip already processed files
    if pseudo_image_filename in processed_files:
        continue

    feature_path = os.path.join(pillar_feature_folder, feature_file)
    
    # ✅ Load Feature Data
    features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

    # ✅ Initialize Empty BEV Grid
    pseudo_image = np.zeros((FEATURE_DIM, GRID_X, GRID_Y), dtype=np.float32)

    # ✅ Map Features to BEV Grid
    for pillar in features:
        x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual pillar coordinates
        pseudo_image[:, x, y] = pillar  # Assign feature vector to BEV cell

    # ✅ Save Pseudo-Image
    np.save(pseudo_image_path, pseudo_image)
    print(f"✅ Saved Pseudo-Image {file_idx+1}: {pseudo_image_path}")

# ✅ Done
print(f"\n🚀 Pseudo-Image Generation Completed for Remaining Files!")


✅ Saved Pseudo-Image 434: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000433.npy
✅ Saved Pseudo-Image 435: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000434.npy
✅ Saved Pseudo-Image 436: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000435.npy
✅ Saved Pseudo-Image 437: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000436.npy
✅ Saved Pseudo-Image 438: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000437.npy
✅ Saved Pseudo-Image 439: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000438.npy
✅ Saved Pseudo-Image 440: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pseudo_images/pseudo_image_000439.npy
✅ Saved Pseudo-Image 441: /home/user/Desktop/noteenv/bin/dataset/kitt

OSError: Not enough free space to write 40960000 bytes

In [1]:
import os
import numpy as np
import torch

# ✅ Define BEV Grid Parameters
GRID_X, GRID_Y = 400, 400  # Adjust based on dataset
FEATURE_DIM = 64  # Output dim from PillarFeatureNet

# ✅ Paths (Only Used for Loading Feature Files)
pillar_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"

# ✅ List All Feature Files
feature_files = [f for f in os.listdir(pillar_feature_folder) if f.endswith('.npy')]
feature_files.sort()

# ✅ Process Feature Files (Without Saving)
for file_idx, feature_file in enumerate(feature_files):
    feature_path = os.path.join(pillar_feature_folder, feature_file)
    
    # ✅ Load Feature Data
    features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

    # ✅ Initialize Empty BEV Grid (Ensure Correct Shape)
    pseudo_image = np.zeros((FEATURE_DIM, GRID_X, GRID_Y), dtype=np.float32)

    # ✅ Map Features to BEV Grid
    for pillar in features:
        if len(pillar) != FEATURE_DIM:
            continue  # Skip invalid data points

        x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual coordinates
        pseudo_image[:, x, y] = pillar  # Assign feature vector to BEV cell

    # ✅ Convert to PyTorch Tensor (Ensure Shape: [64, 400, 400])
    pseudo_image_tensor = torch.tensor(pseudo_image, dtype=torch.float32)

    # ✅ Ensure Shape is Correct
    assert pseudo_image_tensor.shape == (FEATURE_DIM, GRID_X, GRID_Y), \
        f"Error: Expected shape {(FEATURE_DIM, GRID_X, GRID_Y)}, but got {pseudo_image_tensor.shape}"

    print(f"🚀 Processed Pseudo-Image {file_idx+1} with Shape: {pseudo_image_tensor.shape}")

print("\n✅ Pseudo-Image Processing Completed for All Files Without Saving!")


🚀 Processed Pseudo-Image 1 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 2 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 3 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 4 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 5 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 6 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 7 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 8 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 9 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 10 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 11 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 12 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 13 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 14 with Shape: torch.Size([64, 400, 400])
🚀 Processed Pseudo-Image 15 with Shape: torch.Size([64, 400, 400])
🚀 Pr

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import os

# ✅ Define CNN Backbone
class BEVFeatureExtractor(nn.Module):
    def __init__(self, input_channels=64):
        super(BEVFeatureExtractor, self).__init__()
        self.backbone = models.resnet18(pretrained=False)
        
        # ✅ Modify first conv layer to reduce downsampling
        self.backbone.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=1, padding=3, bias=False)
        
        # ✅ Remove downsampling layers to maintain resolution
        self.backbone.maxpool = nn.Identity()  # Remove max pooling
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])  # Keep feature extractor layers

    def forward(self, x):
        return self.backbone(x)  # Expected Output: [1, 512, 50, 50]

# ✅ Initialize Model
device = "cuda" if torch.cuda.is_available() else "cpu"
bev_feature_extractor = BEVFeatureExtractor().to(device)
bev_feature_extractor.eval()  # Set to eval mode for inference

# ✅ Define BEV Grid Parameters
GRID_X, GRID_Y = 400, 400  # Define BEV image size
FEATURE_DIM = 64  # Output dim from PillarFeatureNet

# ✅ Paths
pillar_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"

# ✅ List All Feature Files
feature_files = [f for f in os.listdir(pillar_feature_folder) if f.endswith('.npy')]
feature_files.sort()

# ✅ Process Each Feature File and Extract BEV Features
for file_idx, feature_file in enumerate(feature_files):
    feature_path = os.path.join(pillar_feature_folder, feature_file)

    # ✅ Load Feature Data
    features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

    # ✅ Initialize Empty BEV Grid
    pseudo_image = np.zeros((1, FEATURE_DIM, GRID_X, GRID_Y), dtype=np.float32)

    # ✅ Map Features to BEV Grid
    for pillar in features:
        x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual pillar coordinates
        pseudo_image[0, :, x, y] = pillar  # Assign feature vector to BEV cell

    # ✅ Convert to Torch Tensor
    pseudo_image_tensor = torch.tensor(pseudo_image, dtype=torch.float32).to(device)

    # ✅ Extract BEV Features Using CNN Backbone
    with torch.no_grad():
        bev_feature_map = bev_feature_extractor(pseudo_image_tensor)  # Expected Output: [1, 512, 50, 50]

    print(f"✅ Processed {file_idx+1}/{len(feature_files)} BEV Images | Output Shape: {bev_feature_map.shape}")

# ✅ Done
print(f"\n🚀 BEV Feature Extraction Completed for {len(feature_files)} Files!")


/home/user/Desktop/noteenv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/Desktop/noteenv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Processed 1/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 2/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 3/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 4/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 5/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 6/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 7/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 8/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 9/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 10/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 11/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 12/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 13/5984 BEV Images | Output Shape: torch.Size([1, 512, 50, 50])
✅ Processed 14/5984 B

In [2]:
import torch
import torch.nn as nn

# ✅ Define RPN for 3D Object Detection
class RPN3D(nn.Module):
    def __init__(self, in_channels=512, num_anchors=9):
        super(RPN3D, self).__init__()
        
        # Intermediate CNN for feature processing
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        
        # Prediction Heads
        self.cls_head = nn.Conv2d(256, num_anchors * 2, kernel_size=1)  # Classification (Object/No Object)
        self.reg_head = nn.Conv2d(256, num_anchors * 6, kernel_size=1)  # Regression (x, y, w, h, depth, angle)

    def forward(self, bev_features):
        x = self.relu(self.conv1(bev_features))
        cls_preds = self.cls_head(x)  # Shape: [Batch, num_anchors * 2, H, W]
        reg_preds = self.reg_head(x)  # Shape: [Batch, num_anchors * 6, H, W]
        return cls_preds, reg_preds

# ✅ Initialize RPN Model
rpn_model = RPN3D().to(device)
print(f"✅ RPN Model Ready | Input: [1, 512, 50, 50] | Outputs: Classification & Regression")


✅ RPN Model Ready | Input: [1, 512, 50, 50] | Outputs: Classification & Regression


In [3]:
class RPN3DLoss(nn.Module):
    def __init__(self):
        super(RPN3DLoss, self).__init__()
        self.cls_loss = nn.CrossEntropyLoss()  # Objectness classification loss
        self.reg_loss = nn.SmoothL1Loss()  # Bounding box regression loss

    def forward(self, cls_preds, reg_preds, cls_targets, reg_targets):
        loss_cls = self.cls_loss(cls_preds, cls_targets)  # Class Loss
        loss_reg = self.reg_loss(reg_preds, reg_targets)  # Box Regression Loss
        total_loss = loss_cls + loss_reg  # Combine both losses
        return total_loss

# ✅ Initialize Loss Function
rpn_loss = RPN3DLoss().to(device)


In [7]:
class BEVDataset(Dataset):
    def __init__(self, feature_folder):
        self.feature_files = sorted([os.path.join(feature_folder, f) for f in os.listdir(feature_folder) if f.endswith('.npy')])

    def __len__(self):
        return len(self.feature_files)

    def __getitem__(self, idx):
        feature_path = self.feature_files[idx]
        features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

        # ✅ Create Empty BEV Grid (No batch dim)
        bev_grid = np.zeros((64, 400, 400), dtype=np.float32)

        # ✅ Map Features to Grid
        for pillar in features:
            x, y = np.random.randint(0, 400), np.random.randint(0, 400)  # Replace with actual coordinates
            bev_grid[:, x, y] = pillar

        # ✅ Convert to Torch Tensor & Add Batch Dim
        bev_grid = torch.tensor(bev_grid, dtype=torch.float32).unsqueeze(0)  # Shape: [1, 64, 400, 400]

        # ✅ Generate Dummy Targets (Replace with KITTI labels if available)
        cls_target = torch.tensor(np.random.randint(0, 2, (50, 50)), dtype=torch.float32)  # Class (0/1)
        reg_target = torch.tensor(np.random.randn(4, 50, 50), dtype=torch.float32)  # Regression (x, y, w, h)

        return bev_grid, cls_target, reg_target


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# ✅ Define Training Parameters
num_epochs = 80  # Adjust as needed
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Define Optimizer
optimizer = optim.Adam(list(bev_feature_extractor.parameters()) + list(rpn_model.parameters()), lr=1e-4)

# ✅ Training Loop
for epoch in range(num_epochs):
    bev_feature_extractor.train()
    rpn_model.train()

    total_loss = 0.0

    for bev_features, cls_target, reg_target in data_loader:
        bev_features, cls_target, reg_target = bev_features.to(device), cls_target.to(device), reg_target.to(device)

        # ✅ Fix Shape Issue: Ensure 4D input for CNN
        bev_features = bev_features.squeeze(1)  # [batch, channels, height, width]

        # ✅ Extract BEV Feature Maps
        bev_feature_map = bev_feature_extractor(bev_features)

        # ✅ Forward Pass through RPN
        cls_preds, reg_preds = rpn_model(bev_feature_map)

        # ✅ Compute Loss
        loss = rpn_loss(cls_preds, reg_preds, cls_target, reg_target)
        total_loss += loss.item()

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {total_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save({
    'bev_feature_extractor': bev_feature_extractor.state_dict(),
    'rpn_model': rpn_model.state_dict(),
}, "trained_model.pth")

print("🚀 Training Completed & Model Saved!")


NameError: name 'bev_feature_extractor' is not defined

In [ ]:
# ✅ Define Training Parameters
num_epochs = 50  # Adjust as needed
learning_rate = 1e-4
optimizer = optim.Adam(list(bev_feature_extractor.parameters()) + list(rpn_model.parameters()), lr=learning_rate)

# ✅ Training Loop
for epoch in range(num_epochs):
    bev_feature_extractor.train()
    rpn_model.train()

    total_loss = 0.0

    for bev_features, cls_target, reg_target in data_loader:
        bev_features, cls_target, reg_target = bev_features.to(device), cls_target.to(device), reg_target.to(device)

        # ✅ Fix Shape Issue: Ensure 4D input for CNN
        bev_features = bev_features.squeeze(1)  # [batch, channels, height, width]

        # ✅ Extract BEV Feature Maps
        bev_feature_map = bev_feature_extractor(bev_features)

        # ✅ Forward Pass through RPN
        cls_preds, reg_preds = rpn_model(bev_feature_map)

        # ✅ Compute Loss
        loss = rpn_loss(cls_preds, reg_preds, cls_target, reg_target)
        total_loss += loss.item()

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {total_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save({
    'bev_feature_extractor': bev_feature_extractor.state_dict(),
    'rpn_model': rpn_model.state_dict(),
}, "trained_model.pth")

print("🚀 Training Completed & Model Saved!")


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# ✅ Define Training Parameters
num_epochs = 50  # Adjust as needed
learning_rate = 1e-4
batch_size = 4  # Adjust based on memory capacity

# ✅ Define Optimizer
optimizer = optim.Adam(list(bev_feature_extractor.parameters()) + list(rpn_model.parameters()), lr=learning_rate)

# ✅ Loss Function (Assumed Defined)
def rpn_loss(cls_preds, reg_preds, cls_target, reg_target):
    cls_loss = nn.CrossEntropyLoss()(cls_preds, cls_target)
    reg_loss = nn.MSELoss()(reg_preds, reg_target)
    return cls_loss + reg_loss

# ✅ Training Loop
for epoch in range(num_epochs):
    bev_feature_map.train()
    rpn_model.train()

    total_loss = 0.0

    for batch_idx, (pillar_features, cls_target, reg_target) in enumerate(data_loader):
        # ✅ Convert Pillar Features to BEV Image Format
        pseudo_images = torch.zeros((batch_size, FEATURE_DIM, GRID_X, GRID_Y), dtype=torch.float32).to(device)

        for i in range(len(pillar_features)):
            feature_data = pillar_features[i].numpy()  # Shape: (N, FEATURE_DIM)

            # ✅ Map Features to BEV Grid (Replace with actual coordinate mapping)
            for pillar in feature_data:
                x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual pillar coords
                pseudo_images[i, :, x, y] = torch.tensor(pillar, dtype=torch.float32).to(device)

        # ✅ Forward Pass through BEV Feature Extractor
        bev_feature_map = bev_feature_extractor(pseudo_images)  # Output: [batch_size, 512, 50, 50]

        # ✅ Forward Pass through RPN
        cls_preds, reg_preds = rpn_model(bev_feature_map)

        # ✅ Compute Loss
        loss = rpn_loss(cls_preds, reg_preds, cls_target.to(device), reg_target.to(device))
        total_loss += loss.item()

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {total_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save({
    'bev_feature_extractor': bev_feature_extractor.state_dict(),
    'rpn_model': rpn_model.state_dict(),
}, "trained_model.pth")

print("🚀 Training Completed & Model Saved!")


NameError: name 'bev_feature_extractor' is not defined

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import os

# ✅ Define CNN Backbone
class BEVFeatureExtractor(nn.Module):
    def __init__(self, input_channels=64):
        super(BEVFeatureExtractor, self).__init__()
        self.backbone = models.resnet18(pretrained=False)
        
        # ✅ Modify first conv layer to reduce downsampling
        self.backbone.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=1, padding=3, bias=False)
        
        # ✅ Remove downsampling layers to maintain resolution
        self.backbone.maxpool = nn.Identity()  # Remove max pooling
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])  # Keep feature extractor layers

    def forward(self, x):
        return self.backbone(x)  # Expected Output: [1, 512, 50, 50]

# ✅ Initialize Model
device = "cuda" if torch.cuda.is_available() else "cpu"
bev_feature_extractor = BEVFeatureExtractor().to(device)
bev_feature_extractor.eval()  # Set to eval mode for inference

# ✅ Define BEV Grid Parameters
GRID_X, GRID_Y = 400, 400  # Define BEV image size
FEATURE_DIM = 64  # Output dim from PillarFeatureNet

# ✅ Paths
pillar_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/pillar_features"
output_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"

# ✅ Create Output Directory
os.makedirs(output_folder, exist_ok=True)

# ✅ List All Feature Files
feature_files = [f for f in os.listdir(pillar_feature_folder) if f.endswith('.npy')]
feature_files.sort()

# ✅ Process & Save Each BEV Feature Map
for file_idx, feature_file in enumerate(feature_files):
    feature_path = os.path.join(pillar_feature_folder, feature_file)
    output_path = os.path.join(output_folder, feature_file.replace(".npy", ".pt"))  # Save as .pt file

    # ✅ Skip if already processed
    if os.path.exists(output_path):
        print(f"⚡ Skipping {feature_file} (already processed)")
        continue

    # ✅ Load Feature Data
    features = np.load(feature_path, allow_pickle=True)  # Shape: (N, FEATURE_DIM)

    # ✅ Initialize Empty BEV Grid
    pseudo_image = np.zeros((1, FEATURE_DIM, GRID_X, GRID_Y), dtype=np.float32)

    # ✅ Map Features to BEV Grid
    for pillar in features:
        x, y = np.random.randint(0, GRID_X), np.random.randint(0, GRID_Y)  # Replace with actual pillar coordinates
        pseudo_image[0, :, x, y] = pillar  # Assign feature vector to BEV cell

    # ✅ Convert to Torch Tensor
    pseudo_image_tensor = torch.tensor(pseudo_image, dtype=torch.float32).to(device)

    # ✅ Extract BEV Features Using CNN Backbone
    with torch.no_grad():
        bev_feature_map = bev_feature_extractor(pseudo_image_tensor)  # Output: [1, 512, 50, 50]

    # ✅ Save BEV Feature Map (Efficiently)
    torch.save(bev_feature_map.cpu(), output_path)  # Save as .pt file

    print(f"✅ Processed & Saved {file_idx+1}/{len(feature_files)} | Shape: {bev_feature_map.shape}")

# ✅ Done
print(f"\n🚀 BEV Feature Extraction Completed & Saved for {len(feature_files)} Files!")


/home/user/Desktop/noteenv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/Desktop/noteenv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Processed & Saved 1/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 2/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 3/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 4/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 5/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 6/5984 | Shape: torch.Size([1, 512, 50, 50])
✅ Processed & Saved 7/5984 | Shape: torch.Size([1, 512, 50, 50])


In [4]:
import os
import numpy as np
import torch

# ✅ Paths
output_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"

# ✅ Create Output Directory if not exists
os.makedirs(output_folder, exist_ok=True)

# ✅ Save Processed BEV Features
if 'bev_feature_map' in locals() or 'bev_feature_map' in globals():  # Check if BEV features exist in memory
    for file_idx, feature_file in enumerate(feature_files):  # Use previously loaded feature file names
        output_path = os.path.join(output_folder, feature_file.replace(".npy", "_bev.npy"))

        # ✅ Convert to NumPy and Save
        bev_feature_map_np = bev_feature_map.cpu().numpy()  # Convert to NumPy
        np.save(output_path, bev_feature_map_np)  # Save in .npy format

        print(f"✅ Saved {file_idx+1}/{len(feature_files)} | Shape: {bev_feature_map_np.shape}")

    print(f"\n🚀 All BEV Features Saved Successfully!")
else:
    print("⚠ No processed BEV features found in memory. Please reprocess or load them.")


✅ Saved 1/5984 | Shape: (1, 512, 50, 50)
✅ Saved 2/5984 | Shape: (1, 512, 50, 50)
✅ Saved 3/5984 | Shape: (1, 512, 50, 50)
✅ Saved 4/5984 | Shape: (1, 512, 50, 50)
✅ Saved 5/5984 | Shape: (1, 512, 50, 50)
✅ Saved 6/5984 | Shape: (1, 512, 50, 50)
✅ Saved 7/5984 | Shape: (1, 512, 50, 50)
✅ Saved 8/5984 | Shape: (1, 512, 50, 50)
✅ Saved 9/5984 | Shape: (1, 512, 50, 50)
✅ Saved 10/5984 | Shape: (1, 512, 50, 50)
✅ Saved 11/5984 | Shape: (1, 512, 50, 50)
✅ Saved 12/5984 | Shape: (1, 512, 50, 50)
✅ Saved 13/5984 | Shape: (1, 512, 50, 50)
✅ Saved 14/5984 | Shape: (1, 512, 50, 50)
✅ Saved 15/5984 | Shape: (1, 512, 50, 50)
✅ Saved 16/5984 | Shape: (1, 512, 50, 50)
✅ Saved 17/5984 | Shape: (1, 512, 50, 50)
✅ Saved 18/5984 | Shape: (1, 512, 50, 50)
✅ Saved 19/5984 | Shape: (1, 512, 50, 50)
✅ Saved 20/5984 | Shape: (1, 512, 50, 50)
✅ Saved 21/5984 | Shape: (1, 512, 50, 50)
✅ Saved 22/5984 | Shape: (1, 512, 50, 50)
✅ Saved 23/5984 | Shape: (1, 512, 50, 50)
✅ Saved 24/5984 | Shape: (1, 512, 50, 50)
✅

In [9]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

# ✅ Define Dataset for Loading BEV Features
class BEVDataset(Dataset):
    def __init__(self, bev_feature_folder, label_folder):
        self.bev_feature_files = sorted([f for f in os.listdir(bev_feature_folder) if f.endswith('_bev.npy')])
        self.bev_feature_folder = bev_feature_folder
        self.label_folder = label_folder

    def __len__(self):
        return len(self.bev_feature_files)

    def __getitem__(self, idx):
        feature_path = os.path.join(self.bev_feature_folder, self.bev_feature_files[idx])
        label_path = os.path.join(self.label_folder, self.bev_feature_files[idx].replace('_bev.npy', '_label.npy'))

        # ✅ Load BEV Feature Map
        bev_feature = np.load(feature_path)  # Shape: [1, 512, 50, 50]
        bev_feature = torch.tensor(bev_feature, dtype=torch.float32)

        # ✅ Load Labels (Classification + Regression Targets)
        labels = np.load(label_path)  # Shape: (classification, regression)
        cls_target, reg_target = labels[0], labels[1]

        return bev_feature, torch.tensor(cls_target, dtype=torch.long), torch.tensor(reg_target, dtype=torch.float32)

# ✅ Define Paths
bev_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
label_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"

# ✅ Load Dataset
dataset = BEVDataset(bev_feature_folder, label_folder)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [10]:
import torch.nn as nn

# ✅ Define RPN Model for 3D Object Detection
class RPN3D(nn.Module):
    def __init__(self, input_channels=512):
        super(RPN3D, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 256, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.cls_head = nn.Conv2d(256, 2, kernel_size=1)  # Object/Non-Object Classification
        self.reg_head = nn.Conv2d(256, 7, kernel_size=1)  # 3D Box Regression (x, y, z, w, h, d, θ)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        cls_out = self.cls_head(x)  # [B, 2, H, W]
        reg_out = self.reg_head(x)  # [B, 7, H, W]
        return cls_out, reg_out

# ✅ Initialize RPN Model
device = "cuda" if torch.cuda.is_available() else "cpu"
rpn_model = RPN3D().to(device)


In [11]:



import torch.optim as optim

# ✅ Define Training Parameters
num_epochs = 50
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

optimizer = optim.Adam(rpn_model.parameters(), lr=learning_rate)
cls_criterion = nn.CrossEntropyLoss()
reg_criterion = nn.SmoothL1Loss()

# ✅ Training Loop
for epoch in range(num_epochs):
    rpn_model.train()
    total_cls_loss, total_reg_loss = 0.0, 0.0

    for bev_features, cls_target, reg_target in data_loader:
        bev_features, cls_target, reg_target = bev_features.to(device), cls_target.to(device), reg_target.to(device)

        # ✅ Forward Pass
        cls_preds, reg_preds = rpn_model(bev_features)

        # ✅ Compute Loss
        cls_loss = cls_criterion(cls_preds, cls_target)
        reg_loss = reg_criterion(reg_preds, reg_target)
        loss = cls_loss + reg_loss

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_cls_loss += cls_loss.item()
        total_reg_loss += reg_loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} | CLS Loss: {total_cls_loss:.4f} | REG Loss: {total_reg_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save(rpn_model.state_dict(), "trained_rpn_model.pth")
print("🚀 Training Completed & Model Saved!")


FileNotFoundError: [Errno 2] No such file or directory: '/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2/pillar_features_004778_label.npy'

In [18]:
import os


print(os.path.exists('/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2'))


label_folder = r'/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2'

# Check if label folder exists
if not os.path.exists(label_folder):
    print("❌ Label folder does not exist!")

# Check if label files exist
label_files = sorted([f for f in os.listdir(label_folder) if f.endswith('.npy')])

if len(label_files) == 0:
    print("❌ No label files found in:", label_folder)
else:
    print(f"✅ Found {len(label_files)} label files!")


True
❌ No label files found in: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2


In [19]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

# ✅ Define Dataset for Loading BEV Features
class BEVDataset(Dataset):
    def __init__(self, bev_feature_folder, label_folder):
        self.bev_feature_files = sorted([f for f in os.listdir(bev_feature_folder) if f.endswith('_bev.npy')])
        self.bev_feature_folder = bev_feature_folder
        self.label_folder = label_folder

    def __len__(self):
        return len(self.bev_feature_files)

    def __getitem__(self, idx):
        feature_path = os.path.join(self.bev_feature_folder, self.bev_feature_files[idx])
        label_path = os.path.join(self.label_folder, self.bev_feature_files[idx].replace('_bev.npy', '_label.npy'))

        # ✅ Load BEV Feature Map
        bev_feature = np.load(feature_path)  # Shape: [1, 512, 50, 50]
        bev_feature = torch.tensor(bev_feature, dtype=torch.float32)

        # ✅ Load Labels (Classification + Regression Targets)
        labels = np.load(label_path)  # Shape: (classification, regression)
        cls_target, reg_target = labels[0], labels[1]

        return bev_feature, torch.tensor(cls_target, dtype=torch.long), torch.tensor(reg_target, dtype=torch.float32)

# ✅ Define Paths
bev_feature_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
label_folder = r"/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"

# ✅ Load Dataset
dataset = BEVDataset(bev_feature_folder, label_folder)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [20]:
# ✅ Define Training Parameters
num_epochs = 80
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

optimizer = optim.Adam(rpn_model.parameters(), lr=learning_rate)
cls_criterion = nn.CrossEntropyLoss()
reg_criterion = nn.SmoothL1Loss()


In [22]:
# ✅ Training Loop
for epoch in range(num_epochs):
    rpn_model.train()
    total_cls_loss, total_reg_loss = 0.0, 0.0

    for bev_features, cls_target, reg_target in data_loader:
        bev_features, cls_target, reg_target = bev_features.to(device), cls_target.to(device), reg_target.to(device)

        # ✅ Forward Pass
        cls_preds, reg_preds = rpn_model(bev_features)

        # ✅ Compute Loss
        cls_loss = cls_criterion(cls_preds, cls_target)
        reg_loss = reg_criterion(reg_preds, reg_target)
        loss = cls_loss + reg_loss

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_cls_loss += cls_loss.item()
        total_reg_loss += reg_loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} | CLS Loss: {total_cls_loss:.4f} | REG Loss: {total_reg_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save(rpn_model.state_dict(), "trained_rpn_model.pth")
print("🚀 Training Completed & Model Saved!")


FileNotFoundError: [Errno 2] No such file or directory: '/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2/pillar_features_001651_label.npy'

In [23]:
bev_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"

bev_files = sorted([f.replace(".npy", "") for f in os.listdir(bev_folder) if f.endswith('.npy')])
label_files = sorted([f.replace("_label.npy", "") for f in os.listdir(label_folder) if f.endswith('.npy')])

# Compare missing files
missing_labels = set(bev_files) - set(label_files)
if missing_labels:
    print("❌ Missing label files for:", missing_labels)
else:
    print("✅ All label files are present.")


❌ Missing label files for: {'pillar_features_002651_bev', 'pillar_features_003791_bev', 'pillar_features_002612_bev', 'pillar_features_005081_bev', 'pillar_features_000561_bev', 'pillar_features_004317_bev', 'pillar_features_000900_bev', 'pillar_features_002014_bev', 'pillar_features_002707_bev', 'pillar_features_003006_bev', 'pillar_features_004326_bev', 'pillar_features_000348_bev', 'pillar_features_003941_bev', 'pillar_features_001725_bev', 'pillar_features_004665_bev', 'pillar_features_003393_bev', 'pillar_features_000807_bev', 'pillar_features_000802_bev', 'pillar_features_004700_bev', 'pillar_features_002291_bev', 'pillar_features_005284_bev', 'pillar_features_001713_bev', 'pillar_features_000721_bev', 'pillar_features_001120_bev', 'pillar_features_001467_bev', 'pillar_features_003875_bev', 'pillar_features_005043_bev', 'pillar_features_005739_bev', 'pillar_features_005123_bev', 'pillar_features_003963_bev', 'pillar_features_001529_bev', 'pillar_features_003453_bev', 'pillar_feat

In [26]:
import os
import numpy as np

# Paths
label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"
bev_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
output_label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/processed_labels"

# Create output label directory if not exists
os.makedirs(output_label_folder, exist_ok=True)

# Get BEV feature filenames
bev_files = sorted([f.replace(".npy", "") for f in os.listdir(bev_folder) if f.endswith('.npy')])

# Process each label file
for bev_file in bev_files:
    frame_id = bev_file.replace("pillar_features_", "").replace("_bev", "")  # Extract frame ID
    original_label_path = os.path.join(label_folder, f"{frame_id}.txt")
    output_label_path = os.path.join(output_label_folder, f"pillar_features_{frame_id}_label.npy")

    if os.path.exists(original_label_path):
        # Convert KITTI label format to structured numpy array
        with open(original_label_path, "r") as f:
            lines = f.readlines()
        
        # Parse label data (Convert to classification & regression targets)
        cls_target = []
        reg_target = []
        for line in lines:
            parts = line.strip().split(" ")
            obj_class = parts[0]  # Object class (Car, Pedestrian, etc.)
            bbox_3d = list(map(float, parts[8:]))  # 3D bounding box (x, y, z, h, w, l, ry)

            # Convert class to integer index
            class_mapping = {"Car": 0, "Pedestrian": 1, "Cyclist": 2}
            cls_target.append(class_mapping.get(obj_class, -1))  # -1 for unknown classes
            reg_target.append(bbox_3d)

        # Save as numpy array
        np.save(output_label_path, [np.array(cls_target), np.array(reg_target)])
        print(f"✅ Processed: {output_label_path}")
    else:
        print(f"❌ Missing: {original_label_path}")

print("🚀 Label Processing Completed!")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [27]:
import os
import numpy as np

# Paths
label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"
bev_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
output_label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/processed_labels"

# Create output label directory if not exists
os.makedirs(output_label_folder, exist_ok=True)

# Get BEV feature filenames
bev_files = sorted([f.replace(".npy", "") for f in os.listdir(bev_folder) if f.endswith('.npy')])

# Process each label file
for bev_file in bev_files:
    frame_id = bev_file.replace("pillar_features_", "").replace("_bev", "")  # Extract frame ID
    original_label_path = os.path.join(label_folder, f"{frame_id}.txt")
    output_label_path = os.path.join(output_label_folder, f"pillar_features_{frame_id}_label.npz")

    if os.path.exists(original_label_path):
        # Convert KITTI label format to structured numpy array
        with open(original_label_path, "r") as f:
            lines = f.readlines()
        
        # Parse label data (Convert to classification & regression targets)
        cls_target = []
        reg_target = []
        for line in lines:
            parts = line.strip().split(" ")
            obj_class = parts[0]  # Object class (Car, Pedestrian, etc.)
            bbox_3d = list(map(float, parts[8:]))  # 3D bounding box (x, y, z, h, w, l, ry)

            # Convert class to integer index
            class_mapping = {"Car": 0, "Pedestrian": 1, "Cyclist": 2}
            cls_target.append(class_mapping.get(obj_class, -1))  # -1 for unknown classes
            reg_target.append(bbox_3d)

        # Save as .npz file (Dictionary format)
        np.savez(output_label_path, cls_target=np.array(cls_target, dtype=np.int32), reg_target=np.array(reg_target, dtype=np.float32))

        print(f"✅ Processed: {output_label_path}")
    else:
        print(f"❌ Missing: {original_label_path}")

print("🚀 Label Processing Completed!")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [31]:
import os
import numpy as np

# ✅ Paths
bev_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
label_txt_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/label_2"
output_label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/generated_labels"

# ✅ Create output folder
os.makedirs(output_label_folder, exist_ok=True)

# ✅ Get list of BEV feature files
bev_files = sorted([f for f in os.listdir(bev_folder) if f.endswith('.npy')])

for bev_file in bev_files:
    # Extract the index from the filename (e.g., "pillar_features_000123_bev.npy" → "000123")
    file_index = bev_file.replace("pillar_features_", "").replace("_bev.npy", "")
    
    # Find corresponding label txt file (e.g., "000123.txt")
    label_txt_path = os.path.join(label_txt_folder, f"{file_index}.txt")
    output_label_path = os.path.join(output_label_folder, f"pillar_features_{file_index}_label.npy")

    if not os.path.exists(label_txt_path):
        print(f"❌ Missing label file for: {file_index}. Skipping...")
        continue

    # ✅ Read label file
    cls_target = []
    reg_target = []

    with open(label_txt_path, "r") as f:
        lines = f.readlines()

    if len(lines) == 0:
        print(f"⚠️ Warning: Empty label file {file_index}.txt, skipping...")
        continue  # Skip empty label files

    for line in lines:
        elements = line.strip().split()
        if len(elements) < 15:  # Ensure valid data line
            print(f"⚠️ Skipping invalid line in {file_index}.txt: {line}")
            continue  

        obj_class = elements[0]  # Object type (Car, Pedestrian, etc.)

        # Only process "Car" objects for 3D detection
        if obj_class not in ["Car", "Pedestrian", "Cyclist"]:
            continue

        # Extract bbox 3D parameters (h, w, l, x, y, z, rotation)
        h, w, l = map(float, elements[8:11])
        x, y, z = map(float, elements[11:14])
        rotation = float(elements[14])

        # Convert class name to an integer label
        cls_id = {"Car": 0, "Pedestrian": 1, "Cyclist": 2}.get(obj_class, -1)
        if cls_id == -1:
            continue  # Skip unknown classes

        cls_target.append(cls_id)
        reg_target.append([x, y, z, w, l, h, rotation])  # Store as [x, y, z, w, l, h, rotation]

    if len(cls_target) == 0 or len(reg_target) == 0:
        print(f"⚠️ No valid objects found in {file_index}.txt, skipping...")
        continue

    # ✅ Save label file as numpy array
    np.save(output_label_path, [np.array(cls_target, dtype=np.int32), np.array(reg_target, dtype=np.float32)])
    print(f"✅ Saved: {output_label_path}")

print("🚀 Label Generation Complete!")


⚠️ Skipping invalid line in 000000.txt: 



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 1) + inhomogeneous part.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os

# ✅ Paths
bev_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/bev_features"
label_folder = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/generated_labels"

# ✅ Hyperparameters
batch_size = 4
num_epochs = 10
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Custom Dataset Class
class BEVDataset(Dataset):
    def __init__(self, bev_folder, label_folder):
        self.bev_folder = bev_folder
        self.label_folder = label_folder
        self.bev_files = sorted([f for f in os.listdir(bev_folder) if f.endswith('.npy')])
        
    def __len__(self):
        return len(self.bev_files)
    
    def __getitem__(self, idx):
        bev_filename = self.bev_files[idx]
        file_index = bev_filename.replace("pillar_features_", "").replace("_bev.npy", "")
        
        # Load BEV Features
        bev_path = os.path.join(self.bev_folder, bev_filename)
        bev_feature = np.load(bev_path)  # Shape: (64, 400, 400)
        bev_feature = torch.tensor(bev_feature, dtype=torch.float32)

        # Load Labels
        label_path = os.path.join(self.label_folder, f"pillar_features_{file_index}_label.npz")
        if not os.path.exists(label_path):
            print(f"❌ Missing label file: {label_path}")
            return None

        label_data = np.load(label_path)
        cls_target = torch.tensor(label_data["cls_target"], dtype=torch.long)  # Shape: (N,)
        reg_target = torch.tensor(label_data["reg_target"], dtype=torch.float32)  # Shape: (N, 7)

        return bev_feature, cls_target, reg_target

# ✅ Create Dataset & DataLoader
dataset = BEVDataset(bev_folder, label_folder)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# ✅ Simple RPN Model (Replace with PointPillars RPN)
class RPNModel(nn.Module):
    def __init__(self):
        super(RPNModel, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
        )
        self.cls_head = nn.Linear(256 * 400 * 400, 3)  # 3 classes: Car, Pedestrian, Cyclist
        self.reg_head = nn.Linear(256 * 400 * 400, 7)  # 7 regression params

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        cls_pred = self.cls_head(x)
        reg_pred = self.reg_head(x)
        return cls_pred, reg_pred

# ✅ Initialize Model, Loss Functions, Optimizer
rpn_model = RPNModel().to(device)
cls_criterion = nn.CrossEntropyLoss()
reg_criterion = nn.MSELoss()
optimizer = optim.Adam(rpn_model.parameters(), lr=learning_rate)

# ✅ Training Loop
for epoch in range(num_epochs):
    rpn_model.train()
    total_cls_loss, total_reg_loss = 0.0, 0.0

    for batch in data_loader:
        if batch is None:
            continue

        bev_features, cls_target, reg_target = batch
        bev_features, cls_target, reg_target = bev_features.to(device), cls_target.to(device), reg_target.to(device)

        # ✅ Forward Pass
        cls_preds, reg_preds = rpn_model(bev_features)

        # ✅ Compute Loss
        cls_loss = cls_criterion(cls_preds, cls_target)
        reg_loss = reg_criterion(reg_preds, reg_target)
        loss = cls_loss + reg_loss

        # ✅ Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_cls_loss += cls_loss.item()
        total_reg_loss += reg_loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} | CLS Loss: {total_cls_loss:.4f} | REG Loss: {total_reg_loss:.4f}")

# ✅ Save Model Checkpoint
torch.save(rpn_model.state_dict(), "trained_rpn_model.pth")
print("🚀 Training Completed & Model Saved!")


❌ Missing label file: /home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/generated_labels/pillar_features_002002_label.npz


TypeError: object of type 'NoneType' has no len()

In [1]:
import torch
import torch.nn as nn

class DetectionHead(nn.Module):
    def __init__(self, in_channels=128, num_classes=3, num_anchors=9):
        super(DetectionHead, self).__init__()

        self.num_classes = num_classes
        self.num_anchors = num_anchors

        # Classification head: Predicts object scores for each anchor
        self.cls_head = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, padding=1)

        # Regression head: Predicts bbox offsets for each anchor
        self.reg_head = nn.Conv2d(in_channels, num_anchors * 7, kernel_size=3, padding=1)  # (x, y, z, l, w, h, yaw)

        # Direction classification head: Predicts the direction of objects
        self.dir_head = nn.Conv2d(in_channels, num_anchors * 2, kernel_size=3, padding=1)  # Binary direction classification

    def forward(self, x):
        cls_preds = self.cls_head(x)  # (B, num_anchors * num_classes, H, W)
        reg_preds = self.reg_head(x)  # (B, num_anchors * 7, H, W)
        dir_preds = self.dir_head(x)  # (B, num_anchors * 2, H, W)
        
        return cls_preds, reg_preds, dir_preds

# Example usage
detection_head = DetectionHead()
bev_features = torch.randn(2, 128, 50, 50)  # Output from Backbone
cls_out, reg_out, dir_out = detection_head(bev_features)

print(cls_out.shape)  # (B, num_anchors * num_classes, 50, 50)
print(reg_out.shape)  # (B, num_anchors * 7, 50, 50)
print(dir_out.shape)  # (B, num_anchors * 2, 50, 50)


torch.Size([2, 27, 50, 50])
torch.Size([2, 63, 50, 50])
torch.Size([2, 18, 50, 50])


In [2]:
import torch
import torchvision

def apply_nms_3d(boxes, scores, iou_threshold=0.5):
    """
    Applies Non-Maximum Suppression (NMS) for 3D bounding boxes.

    Args:
        boxes (torch.Tensor): Tensor of shape (N, 7) containing (x, y, z, l, w, h, yaw).
        scores (torch.Tensor): Tensor of shape (N,) containing confidence scores.
        iou_threshold (float): IoU threshold for NMS.

    Returns:
        torch.Tensor: Indices of boxes to keep.
    """

    # Convert 3D boxes to BEV (x, y, l, w) for 2D IoU-based NMS
    bev_boxes = boxes[:, [0, 1, 3, 4]]  # (x, y, l, w)

    # Convert BEV boxes to (x1, y1, x2, y2) format
    bev_boxes_xyxy = torch.cat([
        bev_boxes[:, :2] - bev_boxes[:, 2:] / 2,  # (x_min, y_min)
        bev_boxes[:, :2] + bev_boxes[:, 2:] / 2   # (x_max, y_max)
    ], dim=-1)

    # Apply torchvision NMS (uses 2D IoU)
    keep_indices = torchvision.ops.nms(bev_boxes_xyxy, scores, iou_threshold)

    return keep_indices

# Example usage
boxes = torch.tensor([
    [10, 20, 1, 4, 2, 1.5, 0],  # (x, y, z, l, w, h, yaw)
    [11, 21, 1, 4, 2, 1.5, 0],  # Overlapping box
    [50, 60, 1, 5, 3, 1.5, 1.57] # Distant box
])

scores = torch.tensor([0.9, 0.85, 0.95])  # Confidence scores

keep_indices = apply_nms_3d(boxes, scores, iou_threshold=0.5)
filtered_boxes = boxes[keep_indices]

print("Filtered Boxes:\n", filtered_boxes)


Filtered Boxes:
 tensor([[50.0000, 60.0000,  1.0000,  5.0000,  3.0000,  1.5000,  1.5700],
        [10.0000, 20.0000,  1.0000,  4.0000,  2.0000,  1.5000,  0.0000],
        [11.0000, 21.0000,  1.0000,  4.0000,  2.0000,  1.5000,  0.0000]])


In [3]:
import numpy as np
import open3d as o3d

def voxelize_point_cloud(point_cloud, voxel_size=(0.2, 0.2, 0.2), grid_size=(400, 400, 16)):
    """
    Convert a point cloud to a voxel grid.
    :param point_cloud: (N, 3) array of LiDAR points (x, y, z)
    :param voxel_size: Size of each voxel in meters (x, y, z)
    :param grid_size: Number of voxels along each axis (X, Y, Z)
    :return: Voxelized point cloud representation
    """
    x_min, y_min, z_min = np.min(point_cloud, axis=0)
    x_max, y_max, z_max = np.max(point_cloud, axis=0)
    
    # Compute voxel indices
    voxel_indices = ((point_cloud - [x_min, y_min, z_min]) / voxel_size).astype(int)

    # Clip voxel indices to the grid size
    voxel_indices = np.clip(voxel_indices, 0, np.array(grid_size) - 1)

    return voxel_indices

# ✅ Load a KITTI point cloud sample
point_cloud = np.fromfile("/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/velodyne/000001.bin", dtype=np.float32).reshape(-1, 4)[:, :3]  # (x, y, z)

# ✅ Convert to voxel grid
voxel_indices = voxelize_point_cloud(point_cloud)

# ✅ Print voxel grid coordinates
print("Voxelized Grid (Sample):", voxel_indices[:10])  # Print first 10 voxel indices


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Voxelized Grid (Sample): [[399 389  15]
 [399 390  15]
 [399 389  15]
 [399 390  15]
 [399 391  15]
 [399 392  15]
 [399 390  15]
 [399 390  15]
 [399 390  15]
 [399 390  15]]


In [4]:
def visualize_voxel_grid(point_cloud, voxel_size=0.2):
    """
    Visualize a voxelized point cloud using Open3D.
    :param point_cloud: (N, 3) array of point cloud data
    :param voxel_size: Size of each voxel
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)

    o3d.visualization.draw_geometries([voxel_grid])

# ✅ Visualize voxelized point cloud
visualize_voxel_grid(point_cloud)


In [5]:
import torch
import torch.nn as nn

class SSDDetectionHead(nn.Module):
    def __init__(self, in_channels, num_classes=3, num_anchors=9):
        """
        SSD Detection Head for 3D Object Detection.
        :param in_channels: Number of input feature map channels.
        :param num_classes: Number of object categories (Car, Pedestrian, Cyclist).
        :param num_anchors: Number of anchors per grid cell.
        """
        super(SSDDetectionHead, self).__init__()

        # Classification branch: (B, C, H, W) → (B, num_anchors * num_classes, H, W)
        self.cls_head = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=1)

        # Regression branch: (B, C, H, W) → (B, num_anchors * 7, H, W) for (x, y, z, w, l, h, yaw)
        self.reg_head = nn.Conv2d(in_channels, num_anchors * 7, kernel_size=1)

    def forward(self, bev_features):
        """
        Forward pass of SSD Detection Head.
        :param bev_features: (B, C, H, W) BEV feature map.
        :return: Classification and Regression predictions.
        """
        cls_preds = self.cls_head(bev_features)  # (B, num_anchors * num_classes, H, W)
        reg_preds = self.reg_head(bev_features)  # (B, num_anchors * 7, H, W)

        return cls_preds, reg_preds

# ✅ Example: Define the SSD Detection Head
ssd_head = SSDDetectionHead(in_channels=128)

# ✅ Example: Forward pass on random BEV features
bev_features = torch.randn(2, 128, 50, 50)  # (Batch, Channels, Height, Width)
cls_out, reg_out = ssd_head(bev_features)

# ✅ Print output shapes
print("Classification Output Shape:", cls_out.shape)  # (B, num_anchors * num_classes, H, W)
print("Regression Output Shape:", reg_out.shape)  # (B, num_anchors * 7, H, W)


Classification Output Shape: torch.Size([2, 27, 50, 50])
Regression Output Shape: torch.Size([2, 63, 50, 50])


In [6]:
class SSDLoss(nn.Module):
    def __init__(self):
        super(SSDLoss, self).__init__()
        self.cls_loss_fn = nn.CrossEntropyLoss()
        self.reg_loss_fn = nn.SmoothL1Loss()

    def forward(self, cls_preds, reg_preds, cls_targets, reg_targets):
        """
        Compute SSD loss.
        :param cls_preds: (B, num_anchors * num_classes, H, W) Classification output.
        :param reg_preds: (B, num_anchors * 7, H, W) Regression output.
        :param cls_targets: Ground truth class labels.
        :param reg_targets: Ground truth bounding boxes.
        :return: Total loss (classification + regression).
        """
        cls_loss = self.cls_loss_fn(cls_preds, cls_targets)
        reg_loss = self.reg_loss_fn(reg_preds, reg_targets)

        total_loss = cls_loss + reg_loss
        return total_loss

# ✅ Example: Initialize Loss Function
ssd_loss = SSDLoss()


In [12]:
import torch.optim as optim

# ✅ Define Model, Loss, and Optimizer
ssd_head = SSDDetectionHead(in_channels=128)
loss_fn = SSDLoss()
optimizer = optim.Adam(ssd_head.parameters(), lr=0.001)

# ✅ Example Training Loop
for epoch in range(60):
    optimizer.zero_grad()

    # Simulated data
    bev_features = torch.randn(2, 128, 50, 50)  # (B, C, H, W)
    cls_targets = torch.randint(0, 3, (2, 9 * 3, 50, 50))  # Random class labels
    reg_targets = torch.randn(2, 9 * 7, 50, 50)  # Random regression targets

    # Forward pass
    cls_preds, reg_preds = ssd_head(bev_features)

    # Compute loss
    loss = loss_fn(cls_preds, reg_preds, cls_targets, reg_targets)

    # Backpropagation
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [13]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the voxel features folder path (CHANGE THIS TO YOUR PATH)
VOXEL_FEATURES_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata"  

# Load all saved voxel features
voxel_feature_list = []
for file_name in sorted(os.listdir(VOXEL_FEATURES_PATH)):  # Ensure correct order
    file_path = os.path.join(VOXEL_FEATURES_PATH, file_name)
    voxel_feature = torch.load(file_path)
    voxel_feature_list.append(voxel_feature)

# Stack voxel features into a single tensor
voxel_features = torch.stack(voxel_feature_list).squeeze(1)  # Shape: (Batch, C, H, W)
print("Loaded Voxel Features Shape:", voxel_features.shape)

# Define the 2D CNN Backbone
class Backbone2D(nn.Module):
    def __init__(self, input_channels=64, output_channels=128):
        super(Backbone2D, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, output_channels, kernel_size=3, stride=1, padding=1)

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(output_channels)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        return x

# Instantiate and process the voxel features through CNN
backbone = Backbone2D(input_channels=64, output_channels=128)
bev_features = backbone(voxel_features)  
print("BEV Features Shape:", bev_features.shape)  # Expected (Batch, C, H, W)


/tmp/ipykernel_3413/1385946501.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  voxel_feature = torch.load(file_path)


UnpicklingError: STACK_GLOBAL requires str

In [14]:
import numpy as np

voxel_features = np.random.rand(64, 400, 400)
np.save("/home/user/Desktop/noteenv/bin/dataset/kitti/training/split/training/voxeldata/voxel_000000.npy", voxel_features)


In [18]:
def voxelize(point_cloud, voxel_size=(0.2, 0.2, 4), grid_size=(400, 400, 1)):
    """
    Convert point cloud into voxel pillars.
    """
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    voxel_indices = np.floor((point_cloud[:, :3] - np.min(point_cloud[:, :3], axis=0)) / voxel_size).astype(int)

    voxels = np.zeros(grid_size, dtype=np.float32)
    for idx in range(len(voxel_indices)):
        v_x, v_y, v_z = voxel_indices[idx]
        if 0 <= v_x < grid_size[0] and 0 <= v_y < grid_size[1]:
            voxels[v_x, v_y, v_z] = 1  

    return voxels

# Convert entire dataset
voxel_grids = []
for point_cloud, _ in dataloader:
    for pc in point_cloud:
        voxel_grids.append(voxelize(pc.numpy()))

voxel_grids = torch.tensor(np.array(voxel_grids), dtype=torch.float32)
print("Voxel Grid Shape:", voxel_grids.shape)  # (Batch, D, H, W)


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 174, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 174, in <list

In [19]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define KITTI dataset path
KITTI_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/"  # Update with your dataset location

class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.velo_path = os.path.join(root_dir, 'velodyne')
        self.file_list = [f.split('.')[0] for f in os.listdir(self.velo_path) if f.endswith('.bin')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.velo_path, file_id + '.bin')
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        return torch.tensor(point_cloud, dtype=torch.float32)

# Load the full dataset
dataset = KITTIDataset(root_dir=KITTI_PATH)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)

# Test loading
for batch in dataloader:
    point_clouds = batch
    print("Batch Point Cloud Shape:", point_clouds.shape)
    break


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 142, in collate
    return collate_fn_map[elem_type](batch, collate_fn_map=collate_fn_map)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 213, in collate_tensor_fn
    out = elem.new(storage)

In [20]:
def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-length point clouds.
    Pads all point clouds in a batch to the same number of points.
    """
    point_clouds = [item for item in batch]  # Extract point clouds

    # Find the max number of points in this batch
    max_points = max([pc.shape[0] for pc in point_clouds])

    # Pad all point clouds to the max size
    padded_point_clouds = []
    for pc in point_clouds:
        pad_size = max_points - pc.shape[0]
        padded_pc = torch.cat([pc, torch.zeros((pad_size, 4))], dim=0)  # Pad with zeros
        padded_point_clouds.append(padded_pc)

    return torch.stack(padded_point_clouds)



In [21]:
def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-length point clouds.
    Pads all point clouds in a batch to the same number of points.
    """
    point_clouds = [item for item in batch]  # Extract point clouds

    # Find the max number of points in this batch
    max_points = max([pc.shape[0] for pc in point_clouds])

    # Pad all point clouds to the max size
    padded_point_clouds = []
    for pc in point_clouds:
        pad_size = max_points - pc.shape[0]
        padded_pc = torch.cat([pc, torch.zeros((pad_size, 4))], dim=0)  # Pad with zeros
        padded_point_clouds.append(padded_pc)

    return torch.stack(padded_point_clouds)



In [22]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)


In [23]:
for batch in dataloader:
    point_clouds = batch
    print("Batch Point Cloud Shape:", point_clouds.shape)  # Now all should have the same shape
    break


Batch Point Cloud Shape: torch.Size([2, 125828, 4])


In [24]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define KITTI dataset path
KITTI_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/"  # Update this path

class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.velo_path = os.path.join(root_dir, 'velodyne')
        self.file_list = [f.split('.')[0] for f in os.listdir(self.velo_path) if f.endswith('.bin')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.velo_path, file_id + '.bin')
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        return torch.tensor(point_cloud, dtype=torch.float32)

# Load the full dataset with a custom collate function
def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-length point clouds.
    Pads all point clouds in a batch to the same number of points.
    """
    max_points = max([pc.shape[0] for pc in batch])
    padded_point_clouds = [torch.cat([pc, torch.zeros((max_points - pc.shape[0], 4))], dim=0) for pc in batch]
    return torch.stack(padded_point_clouds)

# Define DataLoader
dataloader = DataLoader(KITTIDataset(root_dir=KITTI_PATH), batch_size=2, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)

# Verify dataset loading
for batch in dataloader:
    point_clouds = batch
    print("Batch Point Cloud Shape:", point_clouds.shape)  # All point clouds should have the same shape
    break


Batch Point Cloud Shape: torch.Size([2, 119205, 4])


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PillarFeatureNet(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64, output_dim=128):
        """
        Pillar Feature Network for feature extraction.
        :param input_dim: Number of input features per pillar (x, y, z, intensity).
        :param hidden_dim: Number of hidden layer features.
        :param output_dim: Output feature size.
        """
        super(PillarFeatureNet, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.bn2 = nn.BatchNorm1d(output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Forward pass of Pillar Feature Net.
        :param x: (Batch, Num_Pillars, Num_Features)
        :return: (Batch, Num_Pillars, Output_Features)
        """
        x = self.relu(self.bn1(self.linear1(x)))
        x = self.relu(self.bn2(self.linear2(x)))
        return x


In [27]:
# Instantiate the network
pillar_net = PillarFeatureNet(input_dim=4, hidden_dim=64, output_dim=128)

# Process all voxel data through Pillar Feature Net
pillar_features = pillar_net(voxel_grids.view(-1, 4))  # Flatten pillars

print("Pillar Features Shape:", pillar_features.shape)  # Expected: (Batch, Num_Pillars, Output_Features)


AttributeError: 'list' object has no attribute 'view'

In [10]:
import os
import shutil
import random

# Define paths
KITTI_BASE = "/home/user/Desktop/noteenv/bin/dataset/kitti/training/"  # Update with your dataset path
TRAIN_DIR = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/"
VAL_DIR = "/home/user/Desktop/noteenv/bin/dataset/kitti/val/"

# Ensure target directories exist
for folder in [TRAIN_DIR, VAL_DIR]:
    os.makedirs(os.path.join(folder, "velodyne"), exist_ok=True)
    os.makedirs(os.path.join(folder, "label_2"), exist_ok=True)

# Get all sample IDs
file_list = sorted([f.split('.')[0] for f in os.listdir(os.path.join(KITTI_BASE, "velodyne")) if f.endswith('.bin')])

# Shuffle for randomness
random.seed(42)  # Ensure reproducibility
random.shuffle(file_list)

# Split 80% Train / 20% Val
split_idx = int(0.8 * len(file_list))
train_files, val_files = file_list[:split_idx], file_list[split_idx:]

# Move files
def move_files(file_ids, src_dir, dest_dir, ext):
    for file_id in file_ids:
        src_file = os.path.join(src_dir, f"{file_id}{ext}")
        dest_file = os.path.join(dest_dir, f"{file_id}{ext}")
        if os.path.exists(src_file):
            shutil.move(src_file, dest_file)

# Move velodyne (point clouds) and labels
move_files(train_files, os.path.join(KITTI_BASE, "velodyne"), os.path.join(TRAIN_DIR, "velodyne"), ".bin")
move_files(train_files, os.path.join(KITTI_BASE, "label_2"), os.path.join(TRAIN_DIR, "label_2"), ".txt")
move_files(val_files, os.path.join(KITTI_BASE, "velodyne"), os.path.join(VAL_DIR, "velodyne"), ".bin")
move_files(val_files, os.path.join(KITTI_BASE, "label_2"), os.path.join(VAL_DIR, "label_2"), ".txt")

print(f"✅ Dataset Split Complete! Train: {len(train_files)}, Validation: {len(val_files)}")


✅ Dataset Split Complete! Train: 5984, Validation: 1497


In [12]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define dataset paths
TRAIN_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/"
VAL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/val/"

class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.velo_path = os.path.join(root_dir, 'velodyne')
        self.label_path = os.path.join(root_dir, 'label_2')
        self.file_list = sorted([f.split('.')[0] for f in os.listdir(self.velo_path) if f.endswith('.bin')])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.velo_path, file_id + '.bin')
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        # Load labels
        label_file = os.path.join(self.label_path, file_id + '.txt')
        labels = self.load_labels(label_file)

        return torch.tensor(point_cloud, dtype=torch.float32), labels

    def load_labels(self, label_file):
        labels = []
        with open(label_file, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split(' ')
                labels.append({
                    'class': parts[0],
                    'bbox': np.array(parts[4:8], dtype=np.float32),
                    'dimensions': np.array(parts[8:11], dtype=np.float32),
                    'location': np.array(parts[11:14], dtype=np.float32),
                    'rotation_y': float(parts[14])
                })
        return labels

# Create train and validation datasets
train_dataset = KITTIDataset(root_dir=TRAIN_PATH)
val_dataset = KITTIDataset(root_dir=VAL_PATH)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4)

# Verify dataset loading
for batch in train_loader:
    point_clouds, labels = batch
    print("Train Batch Point Cloud Shape:", point_clouds.shape)
    break

for batch in val_loader:
    point_clouds, labels = batch
    print("Validation Batch Point Cloud Shape:", point_clouds.shape)
    break


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 317, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 174, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]  # Backwards compatibility.
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 174, in <list

In [13]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define dataset paths
TRAIN_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/"
VAL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/val/"

class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.velo_path = os.path.join(root_dir, 'velodyne')
        self.label_path = os.path.join(root_dir, 'label_2')
        self.file_list = sorted([f.split('.')[0] for f in os.listdir(self.velo_path) if f.endswith('.bin')])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.velo_path, file_id + '.bin')
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        # Load labels
        label_file = os.path.join(self.label_path, file_id + '.txt')
        labels = self.load_labels(label_file)

        return torch.tensor(point_cloud, dtype=torch.float32), labels

    def load_labels(self, label_file):
        labels = []
        with open(label_file, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split(' ')
                labels.append({
                    'class': parts[0],
                    'bbox': np.array(parts[4:8], dtype=np.float32),
                    'dimensions': np.array(parts[8:11], dtype=np.float32),
                    'location': np.array(parts[11:14], dtype=np.float32),
                    'rotation_y': float(parts[14])
                })
        return labels


In [14]:
def custom_collate_fn(batch):
    """
    Custom collate function to handle variable-length point clouds.
    Pads all point clouds in a batch to the same number of points.
    """
    point_clouds = [item[0] for item in batch]  # Extract point clouds
    labels = [item[1] for item in batch]  # Extract labels

    # Find the max number of points in this batch
    max_points = max([pc.shape[0] for pc in point_clouds])

    # Pad all point clouds to the max size
    padded_point_clouds = []
    for pc in point_clouds:
        pad_size = max_points - pc.shape[0]
        padded_pc = torch.cat([pc, torch.zeros((pad_size, 4))], dim=0)  # Pad with zeros
        padded_point_clouds.append(padded_pc)

    return torch.stack(padded_point_clouds), labels


In [15]:
# Create train and validation datasets
train_dataset = KITTIDataset(root_dir=TRAIN_PATH)
val_dataset = KITTIDataset(root_dir=VAL_PATH)

# Create DataLoaders with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=4, collate_fn=custom_collate_fn)

# Verify dataset loading
for batch in train_loader:
    point_clouds, labels = batch
    print("Train Batch Point Cloud Shape:", point_clouds.shape)  # Now all should have the same shape
    break

for batch in val_loader:
    point_clouds, labels = batch
    print("Validation Batch Point Cloud Shape:", point_clouds.shape)  # Now all should have the same shape
    break


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f891c39c9d0><function _MultiProcessingDataLoaderIter.__del__ at 0x7f891c39c9d0>Exception ignored in: 

Traceback (most recent call last):
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    <function _MultiProcessingDataLoaderIter.__del__ at 0x7f891c39c9d0>self._shutdown_workers()
Traceback (most recent call last):

  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
Traceback (most recent call last):
        if w.is_alive():self._shutdown_workers()

  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers
  File "/home/user/Desktop/noteenv/lib/python3.8/site-

Train Batch Point Cloud Shape: torch.Size([2, 121344, 4])


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f891c39c9d0>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f891c39c9d0>Traceback (most recent call last):

  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    Traceback (most recent call last):
self._shutdown_workers()  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    
self._shutdown_workers()  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

    if w.is_alive():  File "/home/user/Desktop/noteenv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1460, in _shutdown_workers

      File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():
      File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_aliv

Validation Batch Point Cloud Shape: torch.Size([2, 115976, 4])


In [16]:
import torch

def voxelize(point_cloud, voxel_size=(0.2, 0.2, 4), grid_size=(400, 400, 1), max_points_per_voxel=32):
    """
    Convert a point cloud into a voxel grid (pillarization).
    
    :param point_cloud: (N, 4) Tensor of point cloud data (x, y, z, intensity).
    :param voxel_size: Size of each voxel in meters.
    :param grid_size: (H, W, D) Grid size in the BEV map.
    :param max_points_per_voxel: Maximum number of points per voxel.
    :return: Voxel grid tensor (H, W, D, max_points_per_voxel, 4).
    """
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    min_coords = torch.min(point_cloud[:, :3], dim=0).values
    voxel_indices = ((point_cloud[:, :3] - min_coords) / torch.tensor(voxel_size)).int()

    # Initialize empty voxel grid
    voxel_grid = torch.zeros((grid_size[0], grid_size[1], grid_size[2], max_points_per_voxel, 4), dtype=torch.float32)
    voxel_counts = torch.zeros((grid_size[0], grid_size[1], grid_size[2]), dtype=torch.int32)

    for i in range(len(voxel_indices)):
        v_x, v_y, v_z = voxel_indices[i]
        if 0 <= v_x < grid_size[0] and 0 <= v_y < grid_size[1] and 0 <= v_z < grid_size[2]:
            count = voxel_counts[v_x, v_y, v_z]
            if count < max_points_per_voxel:
                voxel_grid[v_x, v_y, v_z, count] = point_cloud[i]
                voxel_counts[v_x, v_y, v_z] += 1

    return voxel_grid


In [1]:
import torch

def voxelize(point_cloud, voxel_size=(0.2, 0.2, 4), grid_size=(400, 400, 1), max_points_per_voxel=32):
    """
    Convert a point cloud into a voxel grid (pillarization).
    
    :param point_cloud: (N, 4) Tensor of point cloud data (x, y, z, intensity).
    :param voxel_size: Size of each voxel in meters.
    :param grid_size: (H, W, D) Grid size in the BEV map.
    :param max_points_per_voxel: Maximum number of points per voxel.
    :return: Voxel grid tensor (H, W, D, max_points_per_voxel, 4).
    """
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    min_coords = torch.min(point_cloud[:, :3], dim=0).values
    voxel_indices = ((point_cloud[:, :3] - min_coords) / torch.tensor(voxel_size)).int()

    # Initialize empty voxel grid
    voxel_grid = torch.zeros((grid_size[0], grid_size[1], grid_size[2], max_points_per_voxel, 4), dtype=torch.float32)
    voxel_counts = torch.zeros((grid_size[0], grid_size[1], grid_size[2]), dtype=torch.int32)

    for i in range(len(voxel_indices)):
        v_x, v_y, v_z = voxel_indices[i]
        if 0 <= v_x < grid_size[0] and 0 <= v_y < grid_size[1] and 0 <= v_z < grid_size[2]:
            count = voxel_counts[v_x, v_y, v_z]
            if count < max_points_per_voxel:
                voxel_grid[v_x, v_y, v_z, count] = point_cloud[i]
                voxel_counts[v_x, v_y, v_z] += 1

    return voxel_grid


In [2]:
import os

# Define save paths
TRAIN_VOXEL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train_voxels.pt"
VAL_VOXEL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/val_voxels.pt"

def process_and_save_voxels(data_loader, save_path):
    voxel_grids = []
    
    for point_clouds, _ in data_loader:
        for pc in point_clouds:
            voxel_grids.append(voxelize(pc))
    
    # Convert list to tensor and save
    voxel_grids = torch.stack(voxel_grids)
    torch.save(voxel_grids, save_path)
    print(f"✅ Saved {len(voxel_grids)} voxelized samples to {save_path}")

# Convert and save train & validation voxel grids
process_and_save_voxels(train_loader, TRAIN_VOXEL_PATH)
process_and_save_voxels(val_loader, VAL_VOXEL_PATH)


NameError: name 'train_loader' is not defined

In [2]:
# Import necessary libraries
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define dataset paths
TRAIN_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/"
VAL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/val/"

# Define KITTI Dataset class
class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.velo_path = os.path.join(root_dir, 'velodyne')
        self.label_path = os.path.join(root_dir, 'label_2')
        self.file_list = sorted([f.split('.')[0] for f in os.listdir(self.velo_path) if f.endswith('.bin')])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.velo_path, f"{file_id}.bin")
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        return torch.tensor(point_cloud, dtype=torch.float32)

# Custom collate function to handle variable-length point clouds
def custom_collate_fn(batch):
    max_points = max([pc.shape[0] for pc in batch])
    padded_point_clouds = [torch.cat([pc, torch.zeros((max_points - pc.shape[0], 4))], dim=0) for pc in batch]
    return torch.stack(padded_point_clouds)

# Create DataLoaders
train_dataset = KITTIDataset(root_dir=TRAIN_PATH)
val_dataset = KITTIDataset(root_dir=VAL_PATH)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=0, collate_fn=custom_collate_fn)

print("✅ Train and Validation DataLoaders Initialized Successfully")


✅ Train and Validation DataLoaders Initialized Successfully


In [ ]:
import torch

# Define voxelization function
def voxelize(point_cloud, voxel_size=(0.2, 0.2, 4), grid_size=(400, 400, 1), max_points_per_voxel=32):
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    min_coords = torch.min(point_cloud[:, :3], dim=0).values
    voxel_indices = ((point_cloud[:, :3] - min_coords) / torch.tensor(voxel_size)).int()

    # Initialize empty voxel grid
    voxel_grid = torch.zeros((grid_size[0], grid_size[1], grid_size[2], max_points_per_voxel, 4), dtype=torch.float32)
    voxel_counts = torch.zeros((grid_size[0], grid_size[1], grid_size[2]), dtype=torch.int32)

    for i in range(len(voxel_indices)):
        v_x, v_y, v_z = voxel_indices[i]
        if 0 <= v_x < grid_size[0] and 0 <= v_y < grid_size[1] and 0 <= v_z < grid_size[2]:
            count = voxel_counts[v_x, v_y, v_z]
            if count < max_points_per_voxel:
                voxel_grid[v_x, v_y, v_z, count] = point_cloud[i]
                voxel_counts[v_x, v_y, v_z] += 1

    return voxel_grid

# Define save paths
TRAIN_VOXEL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train_voxels.pt"
VAL_VOXEL_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/val_voxels.pt"

# Convert and save voxel grids with file name display
def process_and_save_voxels(dataset, save_path, dataset_type):
    voxel_grids = []
    total_samples = len(dataset)

    for idx in range(total_samples):
        # Load point cloud from dataset
        point_cloud = dataset[idx]
        voxel_grid = voxelize(point_cloud)

        voxel_grids.append(voxel_grid)

        # Print each file name while processing
        print(f"Processing {dataset_type} File {idx + 1}/{total_samples}: {dataset.file_list[idx]}")

    # Convert list to tensor and save
    voxel_grids = torch.stack(voxel_grids)
    torch.save(voxel_grids, save_path)

    print(f"✅ {dataset_type} voxelized dataset saved! Total files processed: {len(voxel_grids)} → {save_path}")

# Process and save train & validation voxel grids
process_and_save_voxels(train_dataset, TRAIN_VOXEL_PATH, "Train")
process_and_save_voxels(val_dataset, VAL_VOXEL_PATH, "Validation")


Processing Train File 1/5984: 000000
Processing Train File 2/5984: 000001
Processing Train File 3/5984: 000002
Processing Train File 4/5984: 000005
Processing Train File 5/5984: 000006
Processing Train File 6/5984: 000007
Processing Train File 7/5984: 000008
Processing Train File 8/5984: 000010
Processing Train File 9/5984: 000011
Processing Train File 10/5984: 000012
Processing Train File 11/5984: 000014
Processing Train File 12/5984: 000015
Processing Train File 13/5984: 000016
Processing Train File 14/5984: 000018


In [1]:
import torch
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Define Voxelization Function
def voxelize(point_cloud, voxel_size=(0.2, 0.2, 4), grid_size=(400, 400, 1), max_points_per_voxel=32):
    """
    Convert a point cloud into a voxel grid (pillarization).
    
    :param point_cloud: (N, 4) Tensor of point cloud data (x, y, z, intensity).
    :param voxel_size: Size of each voxel in meters.
    :param grid_size: (H, W, D) Grid size in the BEV map.
    :param max_points_per_voxel: Maximum number of points per voxel.
    :return: Voxel grid tensor (H, W, D, max_points_per_voxel, 4).
    """
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    min_coords = torch.min(point_cloud[:, :3], dim=0).values
    voxel_indices = ((point_cloud[:, :3] - min_coords) / torch.tensor(voxel_size)).int()

    # Initialize empty voxel grid
    voxel_grid = torch.zeros((grid_size[0], grid_size[1], grid_size[2], max_points_per_voxel, 4), dtype=torch.float32)
    voxel_counts = torch.zeros((grid_size[0], grid_size[1], grid_size[2]), dtype=torch.int32)

    for i in range(len(voxel_indices)):
        v_x, v_y, v_z = voxel_indices[i]
        if 0 <= v_x < grid_size[0] and 0 <= v_y < grid_size[1] and 0 <= v_z < grid_size[2]:
            count = voxel_counts[v_x, v_y, v_z]
            if count < max_points_per_voxel:
                voxel_grid[v_x, v_y, v_z, count] = point_cloud[i]
                voxel_counts[v_x, v_y, v_z] += 1

    return voxel_grid


In [4]:

import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Define dataset path
DATASET_PATH = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/velodyne/"  # Update this path

# KITTI Dataset Class
class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.file_list = sorted([f.split('.')[0] for f in os.listdir(self.root_dir) if f.endswith('.bin')])

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_id = self.file_list[idx]

        # Load point cloud
        velo_file = os.path.join(self.root_dir, f"{file_id}.bin")
        point_cloud = np.fromfile(velo_file, dtype=np.float32).reshape(-1, 4)  # (x, y, z, intensity)

        return torch.tensor(point_cloud, dtype=torch.float32), file_id

# Initialize Dataset and DataLoader
dataset = KITTIDataset(root_dir=DATASET_PATH)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

print(f"✅ KITTI Dataset Loaded: {len(dataset)} point cloud files")


✅ KITTI Dataset Loaded: 5984 point cloud files


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels=9, out_channels=64):
        """
        Pillar Feature Net (PFN) for feature extraction.
        - in_channels: Number of input features per point (e.g., x, y, z, intensity, etc.).
        - out_channels: Number of output features per pillar.
        """
        super(PillarFeatureNet, self).__init__()

        # Fully connected layers for point-wise feature extraction
        self.fc1 = nn.Linear(in_channels, 64)
        self.fc2 = nn.Linear(64, 64)

    def forward(self, pillars):
        """
        Forward pass for PFN.
        pillars: Tensor of shape (num_pillars, max_points_per_pillar, in_channels)
        
        Output: Tensor of shape (num_pillars, out_channels)
        """
        num_pillars, max_points, in_channels = pillars.shape

        # Normalize: Subtract mean (pillar-wise)
        mean = pillars.mean(dim=1, keepdim=True)  # Compute mean across points in each pillar
        pillars = pillars - mean  # Normalize

        # Apply MLP layers
        features = F.relu(self.fc1(pillars))
        features = F.relu(self.fc2(features))

        # Max-pooling across all points in a pillar
        pillar_features = torch.max(features, dim=1)[0]  # Shape: (num_pillars, out_channels)

        return pillar_features


# **Process voxel data file by file without saving**
def process_voxel_data(dataset_dir, model):
    """
    Load and process voxel dataset line by line.
    
    - dataset_dir: Path to saved voxelized KITTI dataset (.npy files)
    - model: Pillar Feature Net (PFN) model
    
    Displays: Progress line by line.
    """
    voxel_files = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.npy')]

    print(f"Total files to process: {len(voxel_files)}\n")
    
    for idx, file in enumerate(voxel_files):
        voxels = np.load(file)  # Load voxel data
        voxels_tensor = torch.tensor(voxels, dtype=torch.float32)

        # **Fix Shape Issues**
        if len(voxels_tensor.shape) == 2:  # Missing max_points dimension
            voxels_tensor = voxels_tensor.unsqueeze(1)  # Convert to (num_pillars, 1, in_channels)

        # Ensure it has the correct shape (num_pillars, max_points, in_channels)
        if len(voxels_tensor.shape) != 3:
            print(f"Skipping {file} due to incorrect shape: {voxels_tensor.shape}")
            continue

        # Process through PFN (without saving)
        with torch.no_grad():
            pillar_features = model(voxels_tensor)

        print(f"Processed file {idx + 1}/{len(voxel_files)}: {file} -> Extracted shape: {pillar_features.shape}")


# **Run PFN Feature Extraction on KITTI Dataset**
dataset_dir = "/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata"  # Update with your path

# Create PFN Model
pfn = PillarFeatureNet(in_channels=9, out_channels=64)

# Process all voxel files
process_voxel_data(dataset_dir, pfn)


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np

# **Dataset Path**
train_data_dir = '/home/user/Desktop/noteenv/bin/dataset/kitti/train/voxeldata'

# **BEV Grid Parameters**
X_MIN, X_MAX = 0, 70.4  
Y_MIN, Y_MAX = -40, 40  
VOXEL_SIZE = 0.16  
NX = int((X_MAX - X_MIN) / VOXEL_SIZE)  # 440
NY = int((Y_MAX - Y_MIN) / VOXEL_SIZE)  # 500
FEATURE_DIM = 64  

# **CNN Backbone Model (ResNet-Style 2D CNN)**
class BEVBackbone(nn.Module):
    def __init__(self):
        super(BEVBackbone, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(FEATURE_DIM, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.backbone(x)

# **Initialize Model**
bev_backbone = BEVBackbone()

# **Function to Convert to BEV Map**
def create_bev_map(pillar_features, voxel_indices):
    bev_map = torch.zeros((FEATURE_DIM, NX, NY), dtype=torch.float32)  
    for i in range(pillar_features.shape[0]):  
        x_idx, y_idx = voxel_indices[i]  
        if 0 <= x_idx < NX and 0 <= y_idx < NY:  
            bev_map[:, x_idx, y_idx] = pillar_features[i]  
    return bev_map

# **Load and Process Each File**
voxel_files = [f for f in os.listdir(train_data_dir) if f.endswith('.npy')]

for i, file in enumerate(voxel_files):
    file_path = os.path.join(train_data_dir, file)
    
    # **Load Voxel Data**
    voxel_data = np.load(file_path)  # Shape: (num_pillars, 9)
    
    if voxel_data.shape[1] != 9:
        print(f"Skipping {file_path} due to incorrect shape: {voxel_data.shape}")
        continue

    # **Simulated Pillar Feature Extraction** (Replace with PFN model output)
    num_pillars = voxel_data.shape[0]
    pillar_features = torch.rand((num_pillars, FEATURE_DIM))  # Simulated PFN output

    # **Simulated Voxel Indices** (Replace with actual indices)
    voxel_indices = np.random.randint(0, NX, size=(num_pillars, 2))

    # **Convert to BEV Map**
    bev_feature_map = create_bev_map(pillar_features, voxel_indices)

    # **Pass Through CNN Backbone**
    bev_feature_map = bev_feature_map.unsqueeze(0)  # Add batch dimension
    extracted_features = bev_backbone(bev_feature_map)

    # **Display Progress**
    print(f"Processed file {i+1}/{len(voxel_files)}: {file} -> Extracted Features Shape: {extracted_features.shape}")

print("🚀 BEV Feature Extraction + 2D CNN Processing Completed for Entire Dataset!")


Processed file 1/5984: voxel_000808.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 2/5984: voxel_002809.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 3/5984: voxel_004636.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4/5984: voxel_003738.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 5/5984: voxel_000743.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 6/5984: voxel_005488.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 7/5984: voxel_000975.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 8/5984: voxel_003050.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 9/5984: voxel_005693.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 10/5984: voxel_005455.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])

Processed file 83/5984: voxel_000660.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 84/5984: voxel_001746.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 85/5984: voxel_000204.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 86/5984: voxel_001603.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 87/5984: voxel_003108.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 88/5984: voxel_002446.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 89/5984: voxel_005127.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 90/5984: voxel_003192.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 91/5984: voxel_001794.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 92/5984: voxel_001144.npy -> Extracted Features Shape: torch.Size([1, 512, 1

Processed file 164/5984: voxel_003048.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 165/5984: voxel_000260.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 166/5984: voxel_000412.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 167/5984: voxel_004751.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 168/5984: voxel_001670.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 169/5984: voxel_003637.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 170/5984: voxel_003218.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 171/5984: voxel_003344.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 172/5984: voxel_000417.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 173/5984: voxel_005604.npy -> Extracted Features Shape: torch.Size(

Processed file 245/5984: voxel_004716.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 246/5984: voxel_004809.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 247/5984: voxel_001769.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 248/5984: voxel_003356.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 249/5984: voxel_003896.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 250/5984: voxel_003337.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 251/5984: voxel_003874.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 252/5984: voxel_005044.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 253/5984: voxel_002714.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 254/5984: voxel_003042.npy -> Extracted Features Shape: torch.Size(

Processed file 326/5984: voxel_000996.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 327/5984: voxel_004501.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 328/5984: voxel_001062.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 329/5984: voxel_002522.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 330/5984: voxel_000800.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 331/5984: voxel_004508.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 332/5984: voxel_000875.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 333/5984: voxel_003484.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 334/5984: voxel_000740.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 335/5984: voxel_000624.npy -> Extracted Features Shape: torch.Size(

...

Processed file 4494/5984: voxel_003080.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4495/5984: voxel_002195.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4496/5984: voxel_002571.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4497/5984: voxel_004780.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4498/5984: voxel_002988.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4499/5984: voxel_000080.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4500/5984: voxel_002217.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4501/5984: voxel_000886.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4502/5984: voxel_005758.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4503/5984: voxel_002492.npy -> Extracted Features Shape: t

Processed file 4574/5984: voxel_002247.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4575/5984: voxel_003890.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4576/5984: voxel_003542.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4577/5984: voxel_000889.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4578/5984: voxel_004362.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4579/5984: voxel_003053.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4580/5984: voxel_003845.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4581/5984: voxel_003396.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4582/5984: voxel_005092.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4583/5984: voxel_003187.npy -> Extracted Features Shape: t

Processed file 4654/5984: voxel_003680.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4655/5984: voxel_004460.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4656/5984: voxel_002648.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4657/5984: voxel_004267.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4658/5984: voxel_004437.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4659/5984: voxel_002336.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4660/5984: voxel_000757.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4661/5984: voxel_001315.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4662/5984: voxel_003206.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4663/5984: voxel_003300.npy -> Extracted Features Shape: t

Processed file 4734/5984: voxel_000326.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4735/5984: voxel_003180.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4736/5984: voxel_005250.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4737/5984: voxel_004077.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4738/5984: voxel_000035.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4739/5984: voxel_000240.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4740/5984: voxel_001788.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4741/5984: voxel_003573.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4742/5984: voxel_004114.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4743/5984: voxel_002190.npy -> Extracted Features Shape: t

Processed file 4814/5984: voxel_000596.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4815/5984: voxel_002799.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4816/5984: voxel_005435.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4817/5984: voxel_000529.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4818/5984: voxel_003967.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4819/5984: voxel_000060.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4820/5984: voxel_004260.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4821/5984: voxel_000767.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4822/5984: voxel_003995.npy -> Extracted Features Shape: torch.Size([1, 512, 110, 125])
Processed file 4823/5984: voxel_000980.npy -> Extracted Features Shape: t